# Production 1- Data Translation For Storage

## Description:

To enable a program to function each time it runs by using an external and persistent data storage system.<br>There are 2 considerations: The physical storage medium and the format/structure of the data<br>
I have opted to import the CSV and then convert to JSON for manipulation and an SQL database for storage

## Design

Produce a model that shows how the data needs to be restructured to take advantage of the selected format<br>
First, I will perform data analysis, to combine all 3 datasets and then determine a design for the resulting database that uses normalisation to reduce redudancies and enhance performance<br>
I will then produce an entity-relationship diagram of the resulting database structure.

### Data Analysis

Manual Analysis of Data sources: "USER_LOG.csv", "ACTIVITY_LOG.csv", "COMPONENT_CODES.csv"

Start by designing a conceptual normalised schema that combines the three datasets while achieving the highest possible normalised form (likely 3NF). This step ensures:<br>
Data redundancy is minimised.<br>
Relationships between data are clearly defined.<br>
Queries are efficient.<br>
Steps to Normalise the Data<br>
Understand the Structure:<br><br>
USER_LOG records user login activity with anonymised names and timestamps.<br>
ACTIVITY_LOG tracks user actions on various course components.<br>
COMPONENT_CODES maps component names to their respective codes.<br>
Identify Entities:<br>
Users: Represents students in the system.<br>
Activities: Logs actions taken by users on course components.<br>
Components: Lists course components with their respective codes.<br><br>
Define Relationships:<br>
Each user can have multiple activities (one-to-many relationship).<br>
Each activity is associated with a single component code (many-to-one relationship).<br><br>
Create a Normalised Schema:<br>
Users Table:<br>
UserID (Primary Key)<br>
FullName (Anonymised)<br><br>
Activities Table:<br>
ActivityID (Primary Key)<br>
UserID (Foreign Key referencing Users)<br>
ComponentCode (Foreign Key referencing Components)<br>
Action<br>
Target<br><br>
Components Table:<br>
ComponentCode (Primary Key)<br>
ComponentName<br><br>
Diagram Example (ERD)<br>
Users ← (1-to-many) → Activities ← (many-to-1) → Components<br>

## Implementation

A parser is designed based on the resulting ERD specification to read in the original data files and translate them, using JSON, into the normalised schema before sanitising.<br>
The translated and sanitised JSON is then output and saved to an SQL database.<br>
The Pandas API was chosen for this implementation because...

### Load data from CSV Files

In [1]:
import pandas as pd

# Load the CSV files
user_log = pd.read_csv('USER_LOG.csv', nrows=10)
activity_log = pd.read_csv('ACTIVITY_LOG.csv', nrows=10)
component_codes = pd.read_csv('COMPONENT_CODES.csv', nrows=10)

# Preview the data
print(user_log.head(10))
print(activity_log.head(10))
print(component_codes.head(10))


FileNotFoundError: [Errno 2] No such file or directory: 'USER_LOG.csv'

### Combine and Normalise Data

Create Unique IDs:<br>
Assign a UserID to each unique user in USER_LOG.<br>
Use the ComponentCode from COMPONENT_CODES for components.<br><br>
Merge Datasets:<br>
Link ACTIVITY_LOG to COMPONENT_CODES using the Component field.<br>
Link the result to USER_LOG using the anonymized FullName.

In [ ]:
# Step 2: Create a Users Table
# Extract unique users from USER_LOG
users_table = user_log[['User Full Name *Anonymized']].drop_duplicates().reset_index(drop=True)
users_table['UserID'] = users_table.index + 1  # Generate unique UserIDs

# Step 3: Map UserIDs in ACTIVITY_LOG
activity_log = activity_log.merge(users_table, on='User Full Name *Anonymized', how='left')

# Step 4: Map Components to Codes using COMPONENT_CODES
activity_log = activity_log.merge(component_codes, on='Component', how='left')

# Step 5: Create Activities Table
# Add unique ActivityIDs
activity_log['ActivityID'] = activity_log.index + 1

# Normalize columns
activities_table = activity_log[['ActivityID', 'UserID', 'Code', 'Action', 'Target']].rename(
    columns={'Code': 'ComponentCode'}
)

# Step 6: Create Components Table
components_table = component_codes.rename(columns={'Code': 'ComponentCode', 'Component': 'ComponentName'})

# Step 7: Validate Results
print("Users Table:")
print(users_table.head())
print("\nActivities Table:")
print(activities_table.head())
print("\nComponents Table:")
print(components_table.head())


Users Table:
   User Full Name *Anonymized  UserID
0                         129       1
1                          26       2
2                          86       3
3                          94       4
4                         147       5

Activities Table:
   ActivityID  UserID ComponentCode   Action            Target
0           1       1          Cour   Viewed           Content
1           2       2          Quiz  Updated          Response
2           3       2          Quiz   Viewed           Attempt
3           4       3        Assign   Viewed        Assignment
4           5       3        Assign   Viewed  Submission_state

Components Table:
  ComponentName ComponentCode
0        Course          Cour
1          Quiz          Quiz
2    Assignment        Assign
3        System           Sys
4       Lecture          Lect


### Convert to JSON

Convert the normalised data into a JSON structure, preserving its relationships.

In [ ]:
import json

# Convert Users Table to JSON
users_json = users_table.to_json(orient='records')
with open('users.json', 'w') as f:
    f.write(users_json)

# Convert Activities Table to JSON
activities_json = activities_table.to_json(orient='records')
with open('activities.json', 'w') as f:
    f.write(activities_json)

# Convert Components Table to JSON
components_json = components_table.to_json(orient='records')
with open('components.json', 'w') as f:
    f.write(components_json)

# Optional: Print the first few records to confirm
print("Users JSON Sample:", users_json[:200])
print("Activities JSON Sample:", activities_json[:200])
print("Components JSON Sample:", components_json[:200])


Users JSON Sample: [{"User Full Name *Anonymized":129,"UserID":1},{"User Full Name *Anonymized":26,"UserID":2},{"User Full Name *Anonymized":86,"UserID":3},{"User Full Name *Anonymized":94,"UserID":4},{"User Full Name *
Activities JSON Sample: [{"ActivityID":1,"UserID":1,"ComponentCode":"Cour","Action":"Viewed","Target":"Content"},{"ActivityID":2,"UserID":2,"ComponentCode":"Quiz","Action":"Updated","Target":"Response"},{"ActivityID":3,"User
Components JSON Sample: [{"ComponentName":"Course","ComponentCode":"Cour"},{"ComponentName":"Quiz","ComponentCode":"Quiz"},{"ComponentName":"Assignment","ComponentCode":"Assign"},{"ComponentName":"System","ComponentCode":"Sy


### Perform Sanitisation

In [ ]:
# Step 5: Data Sanitization

# Remove rows with missing values in critical columns
activities_table = activities_table.dropna(subset=['UserID', 'ComponentCode', 'Action', 'Target'])

# Standardize string formatting for Actions and Targets
activities_table['Action'] = activities_table['Action'].str.strip().str.capitalize()
activities_table['Target'] = activities_table['Target'].str.strip().str.capitalize()

# Remove duplicates (if any)
users_table = users_table.drop_duplicates()
activities_table = activities_table.drop_duplicates()
components_table = components_table.drop_duplicates()

# Validate cleaned data
print("Cleaned Users Table:")
print(users_table.head())

print("\nCleaned Activities Table:")
print(activities_table.head())

print("\nCleaned Components Table:")
print(components_table.head())


Cleaned Users Table:
   User Full Name *Anonymized  UserID
0                         129       1
1                          26       2
2                          86       3
3                          94       4
4                         147       5

Cleaned Activities Table:
   ActivityID  UserID ComponentCode   Action            Target
0           1       1          Cour   Viewed           Content
1           2       2          Quiz  Updated          Response
2           3       2          Quiz   Viewed           Attempt
3           4       3        Assign   Viewed        Assignment
4           5       3        Assign   Viewed  Submission_state

Cleaned Components Table:
  ComponentName ComponentCode
0        Course          Cour
1          Quiz          Quiz
2    Assignment        Assign
3        System           Sys
4       Lecture          Lect


### Save to SQL Database

In [ ]:
import sqlite3

# Step 6: Save the Data to an SQL Database

# Create a SQLite database connection
conn = sqlite3.connect('university_data.db')

# Save the dataframes as tables in the SQL database
users_table.to_sql('Users', conn, if_exists='replace', index=False)
activities_table.to_sql('Activities', conn, if_exists='replace', index=False)
components_table.to_sql('Components', conn, if_exists='replace', index=False)

# Validate by querying the saved tables
print("\nUsers Table from Database:")
print(pd.read_sql('SELECT * FROM Users LIMIT 5', conn))

print("\nActivities Table from Database:")
print(pd.read_sql('SELECT * FROM Activities LIMIT 5', conn))

print("\nComponents Table from Database:")
print(pd.read_sql('SELECT * FROM Components LIMIT 5', conn))

# Close the database connection
conn.close()



Users Table from Database:
   User Full Name *Anonymized  UserID
0                         129       1
1                          26       2
2                          86       3
3                          94       4
4                         147       5

Activities Table from Database:
   ActivityID  UserID ComponentCode   Action            Target
0           1       1          Cour   Viewed           Content
1           2       2          Quiz  Updated          Response
2           3       2          Quiz   Viewed           Attempt
3           4       3        Assign   Viewed        Assignment
4           5       3        Assign   Viewed  Submission_state

Components Table from Database:
  ComponentName ComponentCode
0        Course          Cour
1          Quiz          Quiz
2    Assignment        Assign
3        System           Sys
4       Lecture          Lect


## Reflection

# Production 2- Interactive GUI

## Description

Develop a GUI to enable users to interact with the data and generaate statistical information and visualisations. Focus on designing the layout and identifying the types of interaction that will be required.

## Design

Produce a set of wireframe designs for the application. Focus on giving a complete view of the applications interface.<br>
Produce a set of integration diagraams (state-machines are a possible option here) to demonstrate 2 or 3 key areas of interaction.

### Wireframe description


By thinking of "views/screens"<br>
<br>
1. Welcome view- A user friendly introduction to the program, describing its purpose and functionality. Perhaps also describing the capabilities and limitations, essentiality modailty of usage, of the program.<br>
This screen is only presented on first opening of the program and should be at the top of but not an explicitly navigable part of the navigation heirarchy 
<br>
Components: (Top down) Logo, Header, Text Field, Button<br><br>

2. Main view- A split screen view thst has left pane and right pane. Left pane contains functions and right pane contains output.<br>
Left pane-<br>
Components: (Top down) Header h1, Descriptive Text Field, Navigation bar (allows navigation to switch between views/tabs in the right psne for specific functionality- Load new data View/tab, View existing data View/tab)<br><br>
Load new data View/tab: <br><br>
Components: Header h2, Descriptive Text Field, Label and url field, browse button, load button<br>
Browse function opens Modal view for browsing file system to select CSV, This modal view has a file selection section and a button to choose the selected file. This button (validates) and closes the modal view and sets the filepath for the chosen file into the Load new data View tab's url field.
Load function- loads CSV from url field into the right pane and clears the url field indicating a new CSV can be added. Imagine a 3 CSV file limit for the purpose of the exercise. Each new loaded CSV is presented in the right pane in its own scrollable section<br>

<br>
Right pane<br> Presents 2 diff views depending on selection in left pane of Load new data View or View existing data View<br>
*Load new data View is selected in left pane*<br>
Components: 1..* (Top down) Header h1, Descriptive Text Field, Header h2 1..*, Table(s) 1..*, Footer, button (translate/normalise)- This button opens new modal view with translated dataset exposing sanitsation functionality- Components: (Top down) Header h1, Descriptive Text Field, Header h2 1..*, Table(s) 1..*, Footer, buttons(pane). Functionality is in footer containing button pane for REMOVE, RENAME, (Fk Knowns, maybe also MERGE, RESHAPE according to Useless Yorkies), SAVE. SAVE  Defines schema and checks against any existing schema before saving. If none exist (no main relation), sets current defined schema and saves as main relation. Each subsequent saving checks against existing schema and ensures data to be saved matches and can be input into main relation (validatoin)<br><br>
*View existing data View is selected in left pane*<br>
Components: (Top down) Header h1, Descriptive Text Field, Header h2 1..*, Table(s) 1..*, Footer , buttons(pane). Functionality in footer containing button pane for REMOVE, RENAME, UNDO, SAVE. REMOVE button uses mouseover events for columns and rows, when column hovered: right click to remove, when row hovered: right click to remove. SAVE press validates before saving.<br><br>
When this right pane view is solided, output functions are revealed in another side section or section before the footer, with buttons to do the stats/graphs. Clicking these buttons pesents modal view with the intended output with a button pane to do shit like export or close view.<br><br>

3. Modal Views:<br>
Modal view for browsing file system for files to input
Modal view for Edit/Update data
Modal view for Output statistics/charts
Modal view for Error catching

## Implementation

Create a first iteration implementation of the interface design. This may include small adaptations that are different from the design but you should aim for at least 90% of the design being similar. That is to say it should be very clear that one (design) is a model of the other (implementation). Avoid designing via coding, you will find the result disjointed. The key to GUI is design on paper first, build second.

Use of Tkinter API for GUI generation.
Follow and adapt design elements from event/page flow, and state change description and diagrams
Define and desscribe structural and interactive elements of each page to inform classes and methods
Use Tkinter API in Python to create classes and methods from description
Focus on creating prototype that can reach all views by expected input, not necessarily performing actual data transfer throught the protoype`

Define mainApp Top level<br>
Initialise imports
Define generic methods like closeFrame, NextFrame, Back, and Generic methods like button  <br>
Define specific FrameClasses like WelcomeFrame, MenuFrame etc

In [ ]:
import tkinter as tk

# Superclass for User Input Controls
class UserInputControls:
    class ButtonWidget:
        @staticmethod
        def buttonSetter(parent, text, command, width=10, height=2, x=0, y=0):
            """
            Static method to add a reusable button to a parent widget.
            :param parent: The parent widget (e.g., a frame or window).
            :param text: The text to display on the button.
            :param command: The function to execute on button click.
            :param width: The width of the button.
            :param height: The height of the button.
            :param x: The x-coordinate for button placement.
            :param y: The y-coordinate for button placement.
            """
            button = tk.Button(parent, text=text, command=command, width=width, height=height)
            button.place(x=x, y=y)
            return button


# Parent class: tk.Frame
class ParentObject(tk.Frame):
    def __init__(self, parent, frameLabel="", designation="Parent"):
        super().__init__(parent)
        self.frameLabel = frameLabel
        self.designation = designation  # Dynamically set the designation
        self.label_text = f"My name is {frameLabel}. I am a {designation}"
        self.label = tk.Label(self, text=self.label_text)
        self.label.pack()

# Child class: ChildFrame
class ChildObject(ParentObject):
    def __init__(self, parent, frameLabel="", width=200, height=200):
        # Set the designation to "Child" and initialize the parent frame
        super().__init__(parent, frameLabel=frameLabel, designation="Child")
        self.parentLabel = parent.frameLabel  # Reference the parent's frameLabel

        # Set explicit dimensions for the frame
        self.config(width=width, height=height)
        self.pack_propagate(False)  # Prevent the frame from shrinking to fit its children

        # Update the label text for the child
        self.label_text = f"My name is {frameLabel}. I am a Child of {self.parentLabel}"
        self.label.config(text=self.label_text)  # Update the existing label's text
   
    
    



# Start Tkinter app
def run_tkinter_app():
    root = tk.Tk()
    root.title("Nested Frames Example")
    root.geometry("400x400")

    # Create the first parent frame
    houser_frame = ParentObject(root, frameLabel="Houser Frame")
    houser_frame.pack(pady=10, padx=10)

    # Create a child frame within the parent frame
    #child_frame = ChildFrame(houser_frame, frameLabel="Dave")
    #child_frame.pack(pady=10, padx=10)

    welcome_frame = ChildObject(houser_frame, frameLabel="WelcomedFrame", width=400, height=150)
    welcome_frame.pack(pady=10, padx=10)

    # Create an object as another child of the first child frame
    grandchild_frame = ChildObject(welcome_frame, frameLabel="Mark", width=400, height=150)
    grandchild_frame.pack(pady=10, padx=10)
    b1 = UserInputControls.ButtonWidget.buttonSetter(
        parent=ParentObject,
        text="popy",
        command="",
        width="",
        height="",
        x="",
        y=""
    )
    grandchildButton = ChildObject(grandchild_frame, frameLabel="GCButton", width=400, height=150)
    grandchildButton = b1
    root.mainloop()

# Run the app
run_tkinter_app()


AttributeError: type object 'ParentObject' has no attribute 'tk'

In [ ]:
##### TIRAMISU

import tkinter as tk

class RootView(tk.Tk):
    """Root view representing the main application window."""

    def __init__(self):
        super().__init__()
        self.title("Dynamic Views and Navigation")
        self.geometry("500x400")# Main windo sizer

        # Create the Houser view as a child of RootView
        self.houser = HouserView(self)
        self.houser.pack(fill="both", expand=True, padx=10, pady=10)

        # Add predefined views dynamically to the Houser view
        self.houser.add_view("WelcomeView", WelcomeView, controller=self)
        self.houser.add_view("MenuView", MenuView, controller=self)
        self.houser.add_view("FileInputView", FileInputView, width=500, height=400, bg="lightblue")

        # Show the initial view
        self.houser.show_view("WelcomeView")

class Section:
    # Superclass to hold similar elements or elements that naturally go together
    #Box properties like size, position
    class NavigationSection:
        #Box properties like size, position
        print()

    class NavBar(tk.Widget):
        print("Lei")
        def __init__(self, text. command,):
            self.menu_button = UserInputControls.Button(self, text="Next", command=self.continue_uh)

class HouserView(tk.Frame):
    """Container for all dynamic views."""

    def __init__(self, parent, width=500, height=400, bg="yellow"):
        super().__init__(parent, width=width, height=height, bg=bg)
        self.parent = parent
        self.views = {}  # To track all dynamic views by name

        self.pack_propagate(False)

    class hea(tk.Label):
        print()

    def add_view(self, view_name, view_class, **kwargs):
        """
        Dynamically add a view to the Houser view.
        :param view_name: The name of the view (used as a key in the views dictionary).
        :param view_class: The class of the view to add.
        :param kwargs: Additional arguments for the view class.
        :return: The created view instance.
        """
        view = view_class(self, **kwargs)
        self.views[view_name] = view
        view.grid(row=0, column=0, sticky="nsew")  # Use grid to stack views
        return view

    def show_view(self, view_name):
        """
        Show the view with the given name.
        :param view_name: The name of the view to display.
        """
        if view_name in self.views:
            self.views[view_name].tkraise()

    def showNextView(self, view_name=None):
        if view_name:
            # Show the specific view if view_name is provided
            if view_name in self.views:
                print(f"View view_name '{view_name}'")
                self.current_view = view_name
                view_names = list(self.views.keys())
                current_index = view_names.index(self.current_view) if self.current_view else -1
                next_index = (current_index + 1) % len(view_names)  # Cycle to the next view
                print(f"View next_index'{next_index}'")
                next_view_name = view_names[next_index]
                print(f"View next_view_name'{next_view_name}'")
                self.views[next_view_name].tkraise()
            
        else:
            print(f"View '{view_name}' does not exist.")


class WelcomeView(tk.Frame):
    """Predefined Welcome View."""

    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller  # Reference to the RootView or controller
        
        print(f"Vince {RootView}")


        # Add a welcome label
        self.label = tk.Label(self, text="Welcome to the Program!")
        self.label.pack(pady=5)

        # Add specific widgets to this view
        self.continue_button = UserInputControls.Button(self, text="Continue", command=self.continue_uh)
        self.continue_button.pack(pady=5)

    def continue_uh(self):
        print("Continue button clicked!")
        #self.controller.houser.show_view("MenuView")
        
        self.controller.houser.showNextView("WelcomeView")


class MenuView(tk.Frame):
    """Predefined Menu View."""

    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        Navbar = Section.NavBar

        # Add a menu label
        self.label = tk.Label(self, text="This is the Menu View.")
        self.label.pack(pady=5)

        # Add specific widgets to this view
        self.menu_button = UserInputControls.Button(self, text="Next", command=self.continue_uh)
        self.menu_button.pack(pady=5)

    

    def continue_uh(self):
        print("Continue button clicked again!")
        #self.controller.houser.show_view("MenuView")
        
        self.controller.houser.showNextView("MenuView")

    def backop(self):
        print("backop")


class FileInputView(tk.Frame):
    """Predefined File Input View."""

    def __init__(self, parent, width=600, height=400, bg="lightblue"):
        super().__init__(parent, width=width, height=height, bg=bg)

        # Prevent the frame from resizing based on child widgets
        self.pack_propagate(False)

        # Add a file input label
        self.label = tk.Label(self, text="This is the File Input View.")
        self.label.pack(pady=5)

        # Add a file input button
        self.file_button = UserInputControls.Button(self, text="Upload File", command=self.on_upload_file)
        self.file_button.pack(pady=5)

    def on_upload_file(self):
        print("File uploaded!")





class UserInputControls:
    """Superclass for reusable input controls."""

    class Button(tk.Button):
        """Button widget with parent referencing."""

        def __init__(self, parent, text="", command=None, **kwargs):
            super().__init__(parent, text=text, command=command, **kwargs)

    

if __name__ == "__main__":
    # Initialize the root view
    root = RootView()

    # Start the application
    root.mainloop()





Vince <class '__main__.RootView'>
Continue button clicked!
View view_name 'WelcomeView'
View next_index'1'
View next_view_name'MenuView'
Continue button clicked again!
View view_name 'MenuView'
View next_index'2'
View next_view_name'FileInputView'
File uploaded!


Version B

In [ ]:
### TIRAMISU2

import tkinter as tk


class RootView(tk.Tk):
    """Root view representing the main application window."""

    def __init__(self):
        super().__init__()
        self.title("Dynamic Views and Navigation")
        self.geometry("500x400")  # Main window size

        # Create the Houser view as a child of RootView
        self.houser = HouserView(self)
        self.houser.pack(fill="both", expand=True, padx=10, pady=10)

        # Add predefined views dynamically to the Houser view
        self.houser.add_view("WelcomeView", WelcomeView, controller=self)
        self.houser.add_view("MenuView", MenuView, controller=self)
        self.houser.add_view("FileInputView", FileInputView, width=500, height=400, bg="lightblue")

        # Show the initial view
        self.houser.show_view("WelcomeView")

    def update_title(self, new_title):
        """Update the title of the window."""
        self.title(new_title)


class HouserView(tk.Frame):
    """Container for all dynamic views."""

    def __init__(self, parent, width=500, height=400, bg="yellow"):
        super().__init__(parent, width=width, height=height, bg=bg)
        self.parent = parent
        self.views = {}  # To track all dynamic views by name
        self.pack_propagate(False)

    def add_view(self, view_name, view_class, **kwargs):
        """
        Dynamically add a view to the Houser view.
        :param view_name: The name of the view (used as a key in the views dictionary).
        :param view_class: The class of the view to add.
        :param kwargs: Additional arguments for the view class.
        :return: The created view instance.
        """
        view = view_class(self, **kwargs)
        self.views[view_name] = view
        view.grid(row=0, column=0, sticky="nsew")  # Use grid to stack views
        return view

    def show_view(self, view_name):
        """
        Show the view with the given name.
        :param view_name: The name of the view to display.
        """
        if view_name in self.views:
            self.views[view_name].tkraise()

            # Map view names to friendly titles
            friendly_titles = {
                "WelcomeView": "Welcome Page",
                "MenuView": "Main Menu",
                "FileInputView": "File Upload Page",
            }
            self.parent.update_title(friendly_titles.get(view_name, view_name))

    def show_next_view(self, current_view_name):
        """
        Show the next view in the sequence.
        :param current_view_name: The current view name.
        """
        view_names = list(self.views.keys())
        if current_view_name in view_names:
            current_index = view_names.index(current_view_name)
            next_index = (current_index + 1) % len(view_names)  # Cycle to the next view
            next_view_name = view_names[next_index]
            self.show_view(next_view_name)


class WelcomeView(tk.Frame):
    """Predefined Welcome View."""

    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller  # Reference to the RootView or controller

        # Add a welcome label
        self.label = tk.Label(self, text="Welcome to the Program!")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.continue_button = tk.Button(self, text="Continue", command=self.continue_to_next_view)
        self.continue_button.pack(pady=5)

    def continue_to_next_view(self):
        print("WelcomeView: Continue button clicked!")
        self.controller.houser.show_next_view("WelcomeView")


class MenuView(tk.Frame):
    """Predefined Menu View."""

    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        # Add a menu label
        self.label = tk.Label(self, text="This is the Menu View.")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.menu_button = tk.Button(self, text="Next", command=self.continue_to_next_view)
        self.menu_button.pack(pady=5)

    def continue_to_next_view(self):
        print("MenuView: Next button clicked!")
        self.controller.houser.show_next_view("MenuView")


class FileInputView(tk.Frame):
    """Predefined File Input View."""

    def __init__(self, parent, width=600, height=400, bg="lightblue"):
        super().__init__(parent, width=width, height=height, bg=bg)

        # Prevent the frame from resizing based on child widgets
        self.pack_propagate(False)

        # Add a file input label
        self.label = tk.Label(self, text="This is the File Input View.")
        self.label.pack(pady=5)

        # Add a file input button
        self.file_button = tk.Button(self, text="Upload File", command=self.on_upload_file)
        self.file_button.pack(pady=5)

    def on_upload_file(self):
        print("FileInputView: File uploaded!")


if __name__ == "__main__":
    # Initialize the root view
    root = RootView()

    # Start the application
    root.mainloop()


WelcomeView: Continue button clicked!
MenuView: Next button clicked!


Version C

In [ ]:
import tkinter as tk
from abc import ABC, abstractmethod


class RootView(tk.Tk):
    """Root view representing the main application window."""

    def __init__(self):
        super().__init__()
        self.title("Dynamic Views and Navigation")
        self.geometry("500x400")  # Main window size

        # Create the Houser view as a child of RootView
        self.houser = HouserView(self)
        self.houser.pack(fill="both", expand=True, padx=10, pady=10)

        # Add predefined views dynamically to the Houser view
        self.houser.add_view("WelcomeView", WelcomeView, controller=self)
        self.houser.add_view("MenuView", MenuView, controller=self)
        self.houser.add_view("FileInputView", FileInputView, width=500, height=400, bg="lightblue")

        # Show the initial view
        self.houser.show_view("WelcomeView")

    def update_title(self, new_title):
        """Update the title of the window."""
        self.title(new_title)


class HouserView(tk.Frame):
    """Container for all dynamic views."""

    def __init__(self, parent, width=500, height=400, bg="yellow"):
        super().__init__(parent, width=width, height=height, bg=bg)
        self.parent = parent
        self.views = {}  # To track all dynamic views by name
        self.pack_propagate(False)

    def add_view(self, view_name, view_class, **kwargs):
        """
        Dynamically add a view to the Houser view.
        :param view_name: The name of the view (used as a key in the views dictionary).
        :param view_class: The class of the view to add.
        :param kwargs: Additional arguments for the view class.
        :return: The created view instance.
        """
        view = view_class(self, **kwargs)
        self.views[view_name] = view
        view.grid(row=0, column=0, sticky="nsew")  # Use grid to stack views
        return view

    def show_view(self, view_name):
        """
        Show the view with the given name.
        :param view_name: The name of the view to display.
        """
        if view_name in self.views:
            self.views[view_name].tkraise()

            # Map view names to friendly titles
            friendly_titles = {
                "WelcomeView": "Welcome Page",
                "MenuView": "Main Menu",
                "FileInputView": "File Upload Page",
            }
            self.parent.update_title(friendly_titles.get(view_name, view_name))

    def show_next_view(self, current_view_name):
        """
        Show the next view in the sequence.
        :param current_view_name: The current view name.
        """
        view_names = list(self.views.keys())
        if current_view_name in view_names:
            current_index = view_names.index(current_view_name)
            next_index = (current_index + 1) % len(view_names)  # Cycle to the next view
            next_view_name = view_names[next_index]
            self.show_view(next_view_name)


class NavigationBar(ABC, tk.Frame):
    """Abstract Navigation Bar."""

    def __init__(self, parent, height=50, bg="gray", **kwargs):
        super().__init__(parent, height=height, bg=bg, **kwargs)
        self.parent = parent
        self.pack_propagate(False)

        # Create the "Back" button
        self.back_button = tk.Button(self, text="Back", command=self.on_back_pressed)
        self.back_button.pack(side="left", padx=10, pady=5)

    @abstractmethod
    def on_back_pressed(self):
        """Define behavior for the back button."""
        pass


class WelcomeView(tk.Frame):
    """Predefined Welcome View."""

    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller  # Reference to the RootView or controller

        # Add a navigation bar
        self.navbar = WelcomeNavBar(self, controller)
        self.navbar.pack(fill="x", side="top")

        # Add a welcome label
        self.label = tk.Label(self, text="Welcome to the Program!")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.continue_button = tk.Button(self, text="Continue", command=self.continue_to_next_view)
        self.continue_button.pack(pady=5)

    def continue_to_next_view(self):
        print("WelcomeView: Continue button clicked!")
        self.controller.houser.show_next_view("WelcomeView")


class WelcomeNavBar(NavigationBar):
    """Navigation bar for Welcome View."""

    def on_back_pressed(self):
        print("WelcomeNavBar: Back button pressed.")
        # Go back to the main RootView or handle as needed


class MenuView(tk.Frame):
    """Predefined Menu View."""

    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        # Add a navigation bar
        self.navbar = MenuNavBar(self, controller)
        self.navbar.pack(fill="x", side="top")

        # Add a menu label
        self.label = tk.Label(self, text="This is the Menu View.")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.menu_button = tk.Button(self, text="Next", command=self.continue_to_next_view)
        self.menu_button.pack(pady=5)

    def continue_to_next_view(self):
        print("MenuView: Next button clicked!")
        self.controller.houser.show_next_view("MenuView")


class MenuNavBar(NavigationBar):
    """Navigation bar for Menu View."""

    def on_back_pressed(self):
        print("MenuNavBar: Back button pressed.")
        # Go back to the previous view or handle as needed


class FileInputView(tk.Frame):
    """Predefined File Input View."""

    def __init__(self, parent, width=600, height=400, bg="lightblue"):
        super().__init__(parent, width=width, height=height, bg=bg)

        # Prevent the frame from resizing based on child widgets
        self.pack_propagate(False)

        # Add a navigation bar
        self.navbar = FileInputNavBar(self)
        self.navbar.pack(fill="x", side="top")

        # Add a file input label
        self.label = tk.Label(self, text="This is the File Input View.")
        self.label.pack(pady=5)

        # Add a file input button
        self.file_button = tk.Button(self, text="Upload File", command=self.on_upload_file)
        self.file_button.pack(pady=5)

    def on_upload_file(self):
        print("FileInputView: File uploaded!")


class FileInputNavBar(NavigationBar):
    """Navigation bar for File Input View."""

    def on_back_pressed(self):
        print("FileInputNavBar: Back button pressed.")
        # Go back to the previous view or handle as needed


if __name__ == "__main__":
    # Initialize the root view
    root = RootView()

    # Start the application
    root.mainloop()


TclError: bad screen distance "."

Version D

In [ ]:
import tkinter as tk
from abc import ABC, abstractmethod


class RootView(tk.Tk):
    """Root view representing the main application window."""

    def __init__(self):
        super().__init__()
        self.title("Dynamic Views and Navigation")
        self.geometry("500x400")  # Main window size

        # Create the Houser view as a child of RootView
        self.houser = HouserView(self)
        self.houser.pack(fill="both", expand=True, padx=10, pady=10)

        # Add predefined views dynamically to the Houser view
        self.houser.add_view("WelcomeView", WelcomeView, controller=self)
        self.houser.add_view("MenuView", MenuView, controller=self)
        self.houser.add_view("FileInputView", FileInputView, width=500, height=400, bg="lightblue")

        # Show the initial view
        self.houser.show_view("WelcomeView")

    def update_title(self, new_title):
        """Update the title of the window."""
        self.title(new_title)


class HouserView(tk.Frame):
    """Container for all dynamic views."""

    def __init__(self, parent, width=500, height=400, bg="yellow"):
        super().__init__(parent, width=width, height=height, bg=bg)
        self.parent = parent
        self.views = {}  # To track all dynamic views by name
        self.pack_propagate(False)

    def add_view(self, view_name, view_class, **kwargs):
        """
        Dynamically add a view to the Houser view.
        :param view_name: The name of the view (used as a key in the views dictionary).
        :param view_class: The class of the view to add.
        :param kwargs: Additional arguments for the view class.
        :return: The created view instance.
        """
        view = view_class(self, **kwargs)
        self.views[view_name] = view
        view.grid(row=0, column=0, sticky="nsew")  # Use grid to stack views
        return view

    def show_view(self, view_name):
        """
        Show the view with the given name.
        :param view_name: The name of the view to display.
        """
        if view_name in self.views:
            self.views[view_name].tkraise()

            # Map view names to friendly titles
            friendly_titles = {
                "WelcomeView": "Welcome Page",
                "MenuView": "Main Menu",
                "FileInputView": "File Upload Page",
            }
            self.parent.update_title(friendly_titles.get(view_name, view_name))

    def show_next_view(self, current_view_name):
        """
        Show the next view in the sequence.
        :param current_view_name: The current view name.
        """
        view_names = list(self.views.keys())
        if current_view_name in view_names:
            current_index = view_names.index(current_view_name)
            next_index = (current_index + 1) % len(view_names)  # Cycle to the next view
            next_view_name = view_names[next_index]
            self.show_view(next_view_name)


class NavigationBar(ABC, tk.Frame):
    """Abstract Navigation Bar."""

    def __init__(self, parent, height=50, bg="gray"):
        # Ensure height is a valid integer
        if not isinstance(height, int) or height <= 0:
            raise ValueError("Height must be a positive integer.")

        super().__init__(parent, height=height, bg=bg)
        self.parent = parent
        self.pack_propagate(False)

        # Create the "Back" button
        self.back_button = tk.Button(self, text="Back", command=self.on_back_pressed)
        self.back_button.pack(side="left", padx=10, pady=5)

    @abstractmethod
    def on_back_pressed(self):
        """Define behavior for the back button."""
        pass


class WelcomeNavBar(NavigationBar):
    """Navigation bar for Welcome View."""

    def on_back_pressed(self):
        print("WelcomeNavBar: Back button pressed.")
        # Implement the behavior for the "Back" button


class WelcomeView(tk.Frame):
    """Predefined Welcome View."""

    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller  # Reference to the RootView or controller

        """ # Add a navigation bar
        self.navbar = WelcomeNavBar(self, height=50, bg="lightgray")
        self.navbar.pack(fill="x", side="top") """

        # Add a welcome label
        self.label = tk.Label(self, text="Welcome to the Program!")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.continue_button = tk.Button(self, text="Continue", command=self.continue_to_next_view)
        self.continue_button.pack(pady=5)

    def continue_to_next_view(self):
        print("WelcomeView: Continue button clicked!")
        self.controller.houser.show_next_view("WelcomeView")


class MenuNavBar(NavigationBar):
    """Navigation bar for Menu View."""

    def on_back_pressed(self):
        print("MenuNavBar: Back button pressed.")
        # Implement the behavior for the "Back" button


class MenuView(tk.Frame):
    """Predefined Menu View."""

    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        # Add a navigation bar
        self.navbar = MenuNavBar(self, height=50, bg="lightgray")
        self.navbar.pack(fill="x", side="top")

        # Add a menu label
        self.label = tk.Label(self, text="This is the Menu View.")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.menu_button = tk.Button(self, text="Next", command=self.continue_to_next_view)
        self.menu_button.pack(pady=5)

    def continue_to_next_view(self):
        print("MenuView: Next button clicked!")
        self.controller.houser.show_next_view("MenuView")


class FileInputNavBar(NavigationBar):
    """Navigation bar for File Input View."""

    def on_back_pressed(self):
        print("FileInputNavBar: Back button pressed.")
        # Implement the behavior for the "Back" button


class FileInputView(tk.Frame):
    """Predefined File Input View."""

    def __init__(self, parent, width=600, height=400, bg="lightblue"):
        super().__init__(parent, width=width, height=height, bg=bg)

        # Prevent the frame from resizing based on child widgets
        self.pack_propagate(False)

        # Add a navigation bar
        self.navbar = FileInputNavBar(self, height=50, bg="lightgray")
        self.navbar.pack(fill="x", side="top")

        # Add a file input label
        self.label = tk.Label(self, text="This is the File Input View.")
        self.label.pack(pady=5)

        # Add a file input button
        self.file_button = tk.Button(self, text="Upload File", command=self.on_upload_file)
        self.file_button.pack(pady=5)

    def on_upload_file(self):
        print("FileInputView: File uploaded!")


if __name__ == "__main__":
    # Initialize the root view
    root = RootView()

    # Start the application
    root.mainloop()


WelcomeNavBar: Back button pressed.


Version E continue_to_next_view(self):" and "def on_back_pressed(self):" are currently being defined for each of WelcomeView, MenuView, FileInputView. Please rewrite so these only need to be written once.

In [ ]:
import tkinter as tk
from abc import ABC, abstractmethod


class RootView(tk.Tk):
    """Root view representing the main application window."""

    def __init__(self):
        super().__init__()
        self.title("Dynamic Views and Navigation")
        self.geometry("500x400")  # Main window size

        # Create the Houser view as a child of RootView
        self.houser = HouserView(self)
        self.houser.pack(fill="both", expand=True, padx=10, pady=10)

        # Add predefined views dynamically to the Houser view
        self.houser.add_view("WelcomeView", WelcomeView, controller=self)
        self.houser.add_view("MenuView", MenuView, controller=self)
        self.houser.add_view("FileInputView", FileInputView, controller=self, width=500, height=400, bg="lightblue")

        # Show the initial view
        self.houser.show_view("WelcomeView")

    def update_title(self, new_title):
        """Update the title of the window."""
        self.title(new_title)


class HouserView(tk.Frame):
    """Container for all dynamic views."""

    def __init__(self, parent, width=500, height=400, bg="yellow"):
        super().__init__(parent, width=width, height=height, bg=bg)
        self.parent = parent
        self.views = {}  # To track all dynamic views by name
        self.pack_propagate(False)

    def add_view(self, view_name, view_class, **kwargs):
        """
        Dynamically add a view to the Houser view.
        :param view_name: The name of the view (used as a key in the views dictionary).
        :param view_class: The class of the view to add.
        :param kwargs: Additional arguments for the view class.
        :return: The created view instance.
        """
        view = view_class(self, **kwargs)
        self.views[view_name] = view
        view.grid(row=0, column=0, sticky="nsew")  # Use grid to stack views
        return view

    def show_view(self, view_name):
        """
        Show the view with the given name.
        :param view_name: The name of the view to display.
        """
        if view_name in self.views:
            self.views[view_name].tkraise()

            # Map view names to friendly titles
            friendly_titles = {
                "WelcomeView": "Welcome Page",
                "MenuView": "Main Menu",
                "FileInputView": "File Upload Page",
            }
            self.parent.update_title(friendly_titles.get(view_name, view_name))

    def show_next_view(self, current_view_name):
        """
        Show the next view in the sequence.
        :param current_view_name: The current view name.
        """
        view_names = list(self.views.keys())
        if current_view_name in view_names:
            current_index = view_names.index(current_view_name)
            next_index = (current_index + 1) % len(view_names)  # Cycle to the next view
            next_view_name = view_names[next_index]
            self.show_view(next_view_name)

    def show_previous_view(self, current_view_name):
        """
        Show the previous view in the sequence.
        :param current_view_name: The current view name.
        """
        view_names = list(self.views.keys())
        if current_view_name in view_names:
            current_index = view_names.index(current_view_name)
            previous_index = (current_index - 1) % len(view_names)  # Cycle to the previous view
            previous_view_name = view_names[previous_index]
            self.show_view(previous_view_name)


class NavigationBar(ABC, tk.Frame):
    """Abstract Navigation Bar."""

    def __init__(self, parent, height=50, bg="gray"):
        super().__init__(parent, height=height, bg=bg)
        self.parent = parent
        self.pack_propagate(False)

        # Create the "Back" button
        self.back_button = tk.Button(self, text="Back", command=self.parent.on_back_pressed)
        self.back_button.pack(side="left", padx=10, pady=5)

    @abstractmethod
    def on_back_pressed(self):
        """Define behavior for the back button."""
        pass


class BaseView(tk.Frame):
    """Base class for all views."""

    def __init__(self, parent, controller, navbar_class=None, **kwargs):
        super().__init__(parent, **kwargs)
        self.controller = controller

        # Add a navigation bar if provided
        if navbar_class:
            self.navbar = navbar_class(self)
            self.navbar.pack(fill="x", side="top")

    def continue_to_next_view(self):
        """Advance to the next view."""
        current_view_name = [name for name, view in self.controller.houser.views.items() if view is self][0]
        print(f"{current_view_name}: Continue button clicked!")
        self.controller.houser.show_next_view(current_view_name)

    def on_back_pressed(self):
        """Handle the 'Back' button."""
        current_view_name = [name for name, view in self.controller.houser.views.items() if view is self][0]
        print(f"{current_view_name}: Back button clacked!")
        self.controller.houser.show_previous_view(current_view_name)


class MenuNavBar(NavigationBar):
    """Navigation bar for Menu View."""

    def on_back_pressed(self):
        print("MenuNavBar: Back button pressed.")


class MenuView(BaseView):
    """Predefined Menu View."""

    def __init__(self, parent, controller):
        super().__init__(parent, controller, navbar_class=MenuNavBar)

        # Add a menu label
        self.label = tk.Label(self, text="This is the Menu View.")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.menu_button = tk.Button(self, text="Next", command=self.continue_to_next_view)
        self.menu_button.pack(pady=5)


class FileInputNavBar(NavigationBar):
    """Navigation bar for File Input View."""

    def on_back_pressed(self):
        print("FileInputNavBar: Back button pressed.")


class FileInputView(BaseView):
    """Predefined File Input View."""

    def __init__(self, parent, controller, width=600, height=400, bg="lightblue"):
        super().__init__(parent, controller, navbar_class=FileInputNavBar, width=width, height=height, bg=bg)

        # Prevent the frame from resizing based on child widgets
        self.pack_propagate(False)

        # Add a file input label
        self.label = tk.Label(self, text="This is the File Input View.")
        self.label.pack(pady=5)

        # Add a file input button
        self.file_button = tk.Button(self, text="Upload File", command=self.on_upload_file)
        self.file_button.pack(pady=5)

    def on_upload_file(self):
        print("FileInputView: File uploaded!")


class WelcomeView(BaseView):
    """Predefined Welcome View (no navigation bar)."""

    def __init__(self, parent, controller):
        super().__init__(parent, controller)

        # Add a welcome label
        self.label = tk.Label(self, text="Welcome to the Program!")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.continue_button = tk.Button(self, text="Continue", command=self.continue_to_next_view)
        self.continue_button.pack(pady=5)


if __name__ == "__main__":
    # Initialize the root view
    root = RootView()

    # Start the application
    root.mainloop()


WelcomeView: Continue button clicked!
MenuView: Back button clacked!


Version F Modify this code to make it not use hardcoded "class MenuNavBar(NavigationBar)", "class MenuView(BaseView)", "class FileInputNavBar(NavigationBar)" but instead, taking "class =MenuView(BaseView)", for example,instead of an explicit MenuNavBar class, there is logic in MenuView(BaseView), that inherits from a generic super navbar object and just overrides or extends (whichever technique is correct here) the methods and attributes of the super object

In [ ]:
from tkinter import * 
from tkinter import messagebox 
from abc import ABC, abstractmethod


class RootView(tk.Tk):
    """Root view representing the main application window."""

    def __init__(self):
        super().__init__()
        self.title("Dynamic Views and Navigation")
        self.geometry("500x400")  # Main window size

        # Create the Houser view as a child of RootView
        self.houser = HouserView(self)
        self.houser.pack(fill="both", expand=True, padx=10, pady=10)

        # Add predefined views dynamically to the Houser view
        self.houser.add_view("WelcomeView", WelcomeView, controller=self)
        self.houser.add_view("MenuView", MenuView, controller=self)
        self.houser.add_view("FileInputView", FileInputView, controller=self, width=500, height=400, bg="lightblue")

        # Show the initial view
        self.houser.show_view("WelcomeView")

    def update_title(self, new_title):
        """Update the title of the window."""
        self.title(new_title)


class HouserView(tk.Frame):
    """Container for all dynamic views."""

    def __init__(self, parent, width=500, height=400, bg="yellow"):
        super().__init__(parent, width=width, height=height, bg=bg)
        self.parent = parent
        self.views = {}  # To track all dynamic views by name
        self.pack_propagate(False)

    def add_view(self, view_name, view_class, **kwargs):
        """
        Dynamically add a view to the Houser view.
        :param view_name: The name of the view (used as a key in the views dictionary).
        :param view_class: The class of the view to add.
        :param kwargs: Additional arguments for the view class.
        :return: The created view instance.
        """
        view = view_class(self, **kwargs)
        self.views[view_name] = view
        view.grid(row=0, column=0, sticky="nsew")  # Use grid to stack views
        return view

    def show_view(self, view_name):
        """
        Show the view with the given name.
        :param view_name: The name of the view to display.
        """
        if view_name in self.views:
            self.views[view_name].tkraise()

            # Map view names to friendly titles
            friendly_titles = {
                "WelcomeView": "Welcome Page",
                "MenuView": "Main Menu",
                "FileInputView": "File Upload Page",
            }
            self.parent.update_title(friendly_titles.get(view_name, view_name))

    def show_next_view(self, current_view_name):
        """
        Show the next view in the sequence.
        :param current_view_name: The current view name.
        """
        view_names = list(self.views.keys())
        if current_view_name in view_names:
            current_index = view_names.index(current_view_name)
            next_index = (current_index + 1) % len(view_names)  # Cycle to the next view
            next_view_name = view_names[next_index]
            self.show_view(next_view_name)

    def show_previous_view(self, current_view_name):
        """
        Show the previous view in the sequence.
        :param current_view_name: The current view name.
        """
        view_names = list(self.views.keys())
        if current_view_name in view_names:
            current_index = view_names.index(current_view_name)
            previous_index = (current_index - 1) % len(view_names)  # Cycle to the previous view
            previous_view_name = view_names[previous_index]
            self.show_view(previous_view_name)


class NavigationBar(tk.Frame):
    """Generic Navigation Bar."""

    def __init__(self, parent, on_back_pressed, height=50, bg="gray"):
        super().__init__(parent, height=height, bg=bg)
        self.parent = parent
        self.pack_propagate(False)

        # Create the "Back" button
        self.back_button = tk.Button(self, text="Back", command=on_back_pressed)
        self.back_button.pack(side="left", padx=10, pady=5)


class BaseView(tk.Frame):
    """Base class for all views."""

    def __init__(self, parent, controller, **kwargs):
        super().__init__(parent, **kwargs)
        self.controller = controller

    def add_navigation_bar(self, custom_on_back_pressed=None):
        """Add a navigation bar with an optional custom back button handler."""
        on_back_pressed = custom_on_back_pressed or self.on_back_pressed
        self.navbar = NavigationBar(self, on_back_pressed)
        self.navbar.pack(fill="x", side="top")

    def continue_to_next_view(self, target):
        """Advance to the next view."""
        current_view_name = [name for name, view in self.controller.houser.views.items() if view is self][0]
        print(f"{current_view_name}: Continue button clicked!")
        self.controller.houser.show_next_view(current_view_name)

    def on_back_pressed(self):
        """Handle the 'Back' button."""
        current_view_name = [name for name, view in self.controller.houser.views.items() if view is self][0]
        print(f"{current_view_name}: Back button clicked!")
        self.controller.houser.show_previous_view(current_view_name)


class AlertView():
    messagebox.askyesno("askyesno", "Find the value?") 


class WelcomeView(BaseView):
    """Predefined Welcome View (no navigation bar)."""

    def __init__(self, parent, controller):
        super().__init__(parent, controller)

        # Add a welcome label
        self.label = tk.Label(self, text="Welcome to the Program!")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.continue_button = tk.Button(self, text="Continue", command=self.continue_to_next_view)
        self.continue_button.pack(pady=5)


class MenuView(BaseView):
    """Predefined Menu View."""

    def __init__(self, parent, controller):
        super().__init__(parent, controller)

        # Add a navigation bar with a custom back handler
        self.add_navigation_bar(custom_on_back_pressed=self.custom_on_back_pressed)

        # Add a menu label
        self.label = tk.Label(self, text="This is the Menu View.")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.menu_button = tk.Button(self, text="Add New Data", command=self.continue_to_next_view)
        self.menu_button.pack(pady=5)

        # Add a button to navigate to the next view
        self.menu_button = tk.Button(self, text="View Existing Data", command=self.continue_to_next_view)
        self.menu_button.pack(pady=5)

    def custom_on_back_pressed(self):
        print("MenuView: Custom back button pressed!")
        self.on_back_pressed()


class FileInputView(BaseView):
    """Predefined File Input View."""

    def __init__(self, parent, controller, width=600, height=400, bg="lightblue"):
        super().__init__(parent, controller, width=width, height=height, bg=bg)

        # Add a navigation bar with the default back handler
        self.add_navigation_bar()

        # Prevent the frame from resizing based on child widgets
        self.pack_propagate(False)

        # Add a file input label
        self.label = tk.Label(self, text="This is the File Input View.")
        self.label.pack(pady=5)

        # Add a file input button
        self.file_button = tk.Button(self, text="Upload File", command=self.on_upload_file)
        self.file_button.pack(pady=5)

    def on_upload_file(self):
        print("FileInputView: File uploaded!")


if __name__ == "__main__":
    # Initialize the root view
    root = RootView()

    # Start the application
    root.mainloop()


WelcomeView: Continue button clicked!
MenuView: Continue button clicked!
FileInputView: File uploaded!


Version G To make the AlertView() display an alert when the self.continue_button in WelcomeView is pressed, you need to integrate the alert logic into the continue_to_next_view or button command. Below is the corrected code to implement this behavior:



In [ ]:
from tkinter import *
from tkinter import messagebox
from abc import ABC, abstractmethod


class RootView(Tk):
    """Root view representing the main application window."""

    def __init__(self):
        super().__init__()
        self.title("Dynamic Views and Navigation")
        self.geometry("500x400")  # Main window size

        # Create the Houser view as a child of RootView
        self.houser = HouserView(self)
        self.houser.pack(fill="both", expand=True, padx=10, pady=10)

        # Add predefined views dynamically to the Houser view
        self.houser.add_view("WelcomeView", WelcomeView, controller=self)
        self.houser.add_view("MenuView", MenuView, controller=self)
        self.houser.add_view("FileInputView", FileInputView, controller=self, width=500, height=400, bg="lightblue")

        # Show the initial view
        self.houser.show_view("WelcomeView")

    def update_title(self, new_title):
        """Update the title of the window."""
        self.title(new_title)


class HouserView(Frame):
    """Container for all dynamic views."""

    def __init__(self, parent, width=500, height=400, bg="yellow"):
        super().__init__(parent, width=width, height=height, bg=bg)
        self.parent = parent
        self.views = {}  # To track all dynamic views by name
        self.pack_propagate(False)

    def add_view(self, view_name, view_class, **kwargs):
        """
        Dynamically add a view to the Houser view.
        :param view_name: The name of the view (used as a key in the views dictionary).
        :param view_class: The class of the view to add.
        :param kwargs: Additional arguments for the view class.
        :return: The created view instance.
        """
        view = view_class(self, **kwargs)
        self.views[view_name] = view
        view.grid(row=0, column=0, sticky="nsew")  # Use grid to stack views
        return view

    def show_view(self, view_name):
        """
        Show the view with the given name.
        :param view_name: The name of the view to display.
        """
        if view_name in self.views:
            self.views[view_name].tkraise()

            # Map view names to friendly titles
            friendly_titles = {
                "WelcomeView": "Welcome Page",
                "MenuView": "Main Menu",
                "FileInputView": "File Upload Page",
            }
            self.parent.update_title(friendly_titles.get(view_name, view_name))

    def show_next_view(self, current_view_name):
        """
        Show the next view in the sequence.
        :param current_view_name: The current view name.
        """
        view_names = list(self.views.keys())
        if current_view_name in view_names:
            current_index = view_names.index(current_view_name)
            next_index = (current_index + 1) % len(view_names)  # Cycle to the next view
            next_view_name = view_names[next_index]
            self.show_view(next_view_name)

    def show_previous_view(self, current_view_name):
        """
        Show the previous view in the sequence.
        :param current_view_name: The current view name.
        """
        view_names = list(self.views.keys())
        if current_view_name in view_names:
            current_index = view_names.index(current_view_name)
            previous_index = (current_index - 1) % len(view_names)  # Cycle to the previous view
            previous_view_name = view_names[previous_index]
            self.show_view(previous_view_name)


class NavigationBar(Frame):
    """Generic Navigation Bar."""

    def __init__(self, parent, on_back_pressed, height=50, bg="gray"):
        super().__init__(parent, height=height, bg=bg)
        self.parent = parent
        self.pack_propagate(False)

        # Create the "Back" button
        self.back_button = Button(self, text="Back", command=on_back_pressed)
        self.back_button.pack(side="left", padx=10, pady=5)


class BaseView(Frame):
    """Base class for all views."""

    def __init__(self, parent, controller, **kwargs):
        super().__init__(parent, **kwargs)
        self.controller = controller

    def add_navigation_bar(self, custom_on_back_pressed=None):
        """Add a navigation bar with an optional custom back button handler."""
        on_back_pressed = custom_on_back_pressed or self.on_back_pressed
        self.navbar = NavigationBar(self, on_back_pressed)
        self.navbar.pack(fill="x", side="top")

    def continue_to_next_view(self, target):
        """Advance to the next view."""
        current_view_name = [name for name, view in self.controller.houser.views.items() if view is self][0]
        print(f"{current_view_name}: Continue button clicked!")
        self.controller.houser.show_next_view(current_view_name)

    def on_back_pressed(self):
        """Handle the 'Back' button."""
        current_view_name = [name for name, view in self.controller.houser.views.items() if view is self][0]
        print(f"{current_view_name}: Back button clicked!")
        self.controller.houser.show_previous_view(current_view_name)


class AlertView:
    """Alert View to show a interactive dialog."""

    @staticmethod
    def show_error():
        response = messagebox.showerror("showerror", "Error")
        print(f"Alert response: {response}")
        return response
    
    @staticmethod
    def show_query():
        response = messagebox.askyesno("Confirmation", "Diddydoodat?")
        print(f"Alert response: {response}")
        return response

class ModalView:
    """Alert View to show an interactive dialog."""

    @staticmethod
    def show_browser():
        response = messagebox.askyesno("Confirmation", "Find the file")
        print(f"Browser: {response}")
        return response
    
    @staticmethod
    def show_editor():
        response = messagebox.showinfo("showinfo", "Editor")
        print(f"Field Editor: {response}")
        return response




class WelcomeView(BaseView):
    """Predefined Welcome View (no navigation bar)."""

    def __init__(self, parent, controller):
        super().__init__(parent, controller)

        # Add a welcome label
        self.label = Label(self, text="Welcome to the Program!")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.continue_button = Button(self, text="Continue", command=self.on_continue_clicked)
        self.continue_button.pack(pady=5)

    def on_continue_clicked(self):
        """Handle the 'Continue' button click."""
        # Show the alert dialog
        response = AlertView.show_alert()

        # If the user clicks "Yes", proceed to the next view
        if response:
            self.continue_to_next_view("MenuView")  # Replace with the target view name


class MenuView(BaseView):
    """Predefined Menu View."""

    def __init__(self, parent, controller):
        super().__init__(parent, controller)

        # Add a navigation bar with a custom back handler
        self.add_navigation_bar(custom_on_back_pressed=self.custom_on_back_pressed)

        # Add a menu label
        self.label = Label(self, text="This is the Menu View.")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.menu_button = Button(self, text="Add New Data", command=self.on_continue_clicked)
        self.menu_button.pack(pady=5)

        # Add another button
        self.view_button = Button(self, text="View Existing Data", command=self.continue_to_next_view)
        self.view_button.pack(pady=5)
        self.continue_to_next_view("DataOutputView")  # Replace with the target view name
    
    def on_continue_clicked(self):
        """Handle the 'Continue' button click."""
        # Show the alert dialog
        response = AlertView.show_alert()

        # If the user clicks "Yes", proceed to the next view
        if response:
            self.continue_to_next_view("FileInputView")  # Replace with the target view name

    def custom_on_back_pressed(self):
        print("MenuView: Custom back button pressed!")
        self.on_back_pressed()


class FileInputView(BaseView):
    """Predefined File Input View."""

    def __init__(self, parent, controller, width=600, height=400, bg="lightblue"):
        super().__init__(parent, controller, width=width, height=height, bg=bg)

        # Add a navigation bar with the default back handler
        self.add_navigation_bar()

        # Prevent the frame from resizing based on child widgets
        self.pack_propagate(False)

        # Add a file input label
        self.label = Label(self, text="This is the File Input View.")
        self.label.pack(pady=5)

        # Add a file input button
        self.file_button = Button(self, text="Upload File", command=self.on_upload_file)
        self.file_button.pack(pady=5)

    def on_upload_file(self):
        print("FileInputView: File uploaded!")
        # Show the Modal dialog
        response = ModalView.show_browser()

        # If the user clicks "Yes", proceed to the next view
        if response:
            self.continue_to_next_view("FileInputView")  # Replace with the target view name


class DataOutputView(BaseView):
    """Predefined File Input View."""

    def __init__(self, parent, controller, width=600, height=400, bg="lightblue"):
        super().__init__(parent, controller, width=width, height=height, bg=bg)

        # Add a navigation bar with the default back handler
        self.add_navigation_bar()

        # Prevent the frame from resizing based on child widgets
        self.pack_propagate(False)

        # Add label
        self.label = Label(self, text="This is the Data Output Input View.")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.continue_button = Button(self, text="Continue", command=self.on_continue_clicked)
        self.continue_button.pack(pady=5)

    def on_continue_clicked(self):
        """Handle the 'Continue' button click."""
        # Show the alert dialog
        response = AlertView.show_alert()

        # If the user clicks "Yes", proceed to the next view
        if response:
            self.continue_to_next_view("StatsVisMethodsView")  # Replace with the target view name

if __name__ == "__main__":
    # Initialize the root view
    root = RootView()

    # Start the application
    root.mainloop()


Version H. Getting weird


In [ ]:
from tkinter import *
from tkinter import messagebox


class RootView(Tk):
    """Root view representing the main application window."""

    def __init__(self):
        super().__init__()
        self.title("Dynamic Views and Navigation")
        self.geometry("500x400")  # Main window size
        self.sender = ""

        # Create the Houser view as a child of RootView
        self.houser = HouserView(self)
        self.houser.pack(fill="both", expand=True, padx=10, pady=10)

        # Add predefined views dynamically to the Houser view
        self.houser.add_view("WelcomeView", WelcomeView, controller=self)
        self.houser.add_view("MenuView", MenuView, controller=self)
        self.houser.add_view("FileInputView", FileInputView, controller=self, width=500, height=400, bg="lightblue")
        self.houser.add_view("DataOutputView", DataOutputView, controller=self, width=500, height=400, bg="lightgreen")
        self.houser.add_view("StatsVisMethodsView", StatsVisMethodsView, controller=self, width=500, height=400, bg="lightyellow")

        # Show the initial view
        self.houser.show_view("WelcomeView")

    def update_title(self, new_title):
        """Update the title of the window."""
        self.title(new_title)


class HouserView(Frame):
    """Container for all dynamic views."""

    def __init__(self, parent, width=500, height=400, bg="yellow"):
        super().__init__(parent, width=width, height=height, bg=bg)
        self.parent = parent
        self.views = {}  # To track all dynamic views by name
        self.pack_propagate(False)

    def add_view(self, view_name, view_class, **kwargs):
        """
        Dynamically add a view to the Houser view.
        :param view_name: The name of the view (used as a key in the views dictionary).
        :param view_class: The class of the view to add.
        :param kwargs: Additional arguments for the view class.
        :return: The created view instance.
        """
        view = view_class(self, **kwargs)
        self.views[view_name] = view
        view.grid(row=0, column=0, sticky="nsew")  # Use grid to stack views
        return view

    def show_view(self, view_name):
        """
        Show the view with the given name.
        :param view_name: The name of the view to display.
        """
        if view_name in self.views:
            self.views[view_name].tkraise()

            # Map view names to friendly titles
            friendly_titles = {
                "WelcomeView": "Welcome Page",
                "MenuView": "Main Menu",
                "FileInputView": "File Upload Page",
                "DataOutputView": "Data Output Page",
                "StatsVisMethodsView": "Statistics/Visualisations Methods Page"
            }
            self.parent.update_title(friendly_titles.get(view_name, view_name))


class NavigationBar(Frame):
    """Generic Navigation Bar."""

    def __init__(self, parent, on_back_pressed, height=50, bg="gray"):
        super().__init__(parent, height=height, bg=bg)
        self.parent = parent
        self.pack_propagate(False)

        # Create the "Back" button
        self.back_button = Button(self, text="Back", command=on_back_pressed)
        self.back_button.pack(side="left", padx=10, pady=5)


class BaseView(Frame):
    """Base class for all views."""

    def __init__(self, parent, controller, **kwargs):
        super().__init__(parent, **kwargs)
        self.controller = controller

    def add_navigation_bar(self, custom_on_back_pressed=None):
        """Add a navigation bar with an optional custom back button handler."""
        on_back_pressed = custom_on_back_pressed or self.on_back_pressed
        self.navbar = NavigationBar(self, on_back_pressed)
        self.navbar.pack(fill="x", side="top")

    def on_back_pressed(self):
        """Handle the 'Back' button."""
        self.controller.houser.show_view("WelcomeView")######HEY HEY YO need to make show previous page in list/stack or dyrnamically
        #the above is force back press to jump to WelcomeView


class AlertView:
    """Alert View to show a interactive dialog."""

    @staticmethod
    def show_error():
        response = messagebox.showerror("showerror", "Error")
        print(f"Alert response: {response}")
        return response
    
    @staticmethod
    def show_query():
        response = messagebox.askyesno("Confirmation", "Doody?")
        print(f"Alert response: {response}")
        return response

class ModalView:
    """Alert View to show an interactive dialog."""

    @staticmethod
    def show_browser():
        response = messagebox.askyesno("Confirmation", "Find the file")
        print(f"Browser: {response}")
        return response
    
    @staticmethod
    def show_editor():
        response = messagebox.showinfo("showinfo", "Editor")
        print(f"Field Editor: {response}")
        return response
    
    @staticmethod
    def show_stat_vis_output():
        response = messagebox.showinfo("showinfo", "Editor")
        print(f"Output Viewer: {response}")
        return response




class WelcomeView(BaseView):
    """Predefined Welcome View."""

    def __init__(self, parent, controller):
        super().__init__(parent, controller)

        # Add a welcome label
        self.label = Label(self, text="Welcome to the Program!")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.continue_button = Button(self, text="Continue", command=self.on_continue_clicked)
        self.continue_button.pack(pady=5)

    def on_continue_clicked(self):
        """Handle the 'Continue' button click."""
        # Show the alert dialog
        response = AlertView.show_query()

        # If the user clicks "Yes", proceed to the next view
        if response:
            self.controller.houser.show_view("MenuView")


class MenuView(BaseView):
    """Predefined Menu View."""

    def __init__(self, parent, controller):
        super().__init__(parent, controller)

        # Add a navigation bar
        self.add_navigation_bar()

        # Add a menu label
        self.label = Label(self, text="This is the Menu View.")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.FImenu_button = Button(self, text="Go to File Input", command=self.on_FImenu_button_clicked)
        self.FImenu_button.pack(pady=5)

         # Add a button to navigate to the next view
        self.DVmenu_button = Button(self, text="Go to Data Viewer", command=self.on_DVmenu_button_clicked)
        self.DVmenu_button.pack(pady=5)

    def on_FImenu_button_clicked(self):
        """Handle the 'Continue' button click."""
        # Show the alert dialog
        response = AlertView.show_query()
        print(f"smell {response}")
        # If the user clicks "Yes", proceed to the next view
        if response:
            self.controller.houser.show_view("FileInputView")

    def on_DVmenu_button_clicked(self):
        """Handle the 'Continue' button click."""
        self.controller.houser.show_view("DataOutputView")


class FileInputView(BaseView):
    """Predefined File Input View."""

    def __init__(self, parent, controller, width=600, height=400, bg="lightblue"):
        super().__init__(parent, controller, width=width, height=height, bg=bg)

        # Add a navigation bar
        self.add_navigation_bar()

        # Add a file input label
        self.label = Label(self, text="This is the File Input View.")
        self.label.pack(pady=5)

        # Add a file input button
        self.file_button = Button(self, text="Upload File", command=self.openBrowser)
        self.file_button.pack(pady=5)

    def openBrowser(self):
        # Show the modal dialog
        response = ModalView.show_browser()
        if response:
            print("File uploaded!")
            # Navigate to the DataOutputView after file upload
            self.controller.houser.show_view("DataOutputView")
        else:
            print("File NOT uploaded!")


class DataOutputView(BaseView):
    """Data Output View."""

    def __init__(self, parent, controller, width=600, height=400, bg="lightgreen"):
        super().__init__(parent, controller, width=width, height=height, bg=bg)

        # Add a navigation bar
        self.add_navigation_bar()

        # Add a data output label
        self.label = Label(self, text="This is the Data Output View.")
        self.label.pack(pady=5)

        # Add a button to Perform Data acrobatix- load Stats/VisMethods
        self.data_acrobatics_button = Button(self, text="Perform Data acrobatix", command=self.showStatsTools)
        self.data_acrobatics_button.pack(pady=5)

    def showStatsTools(self):
            self.controller.houser.show_view("StatsVisMethodsView")
            print("Klugy")

class StatsVisMethodsView(BaseView):
    """StatsVisMethods View."""

    def __init__(self, parent, controller, width=600, height=400, bg="lightgreen"):
        super().__init__(parent, controller, width=width, height=height, bg=bg)

        # Add a navigation bar
        self.add_navigation_bar()

        # Add a data output label
        self.label = Label(self, text="This is the StatsVisMethods View.")
        self.label.pack(pady=5)

        # Add a button to Perform Data acrobatix- load Stats/VisMethods
        self.back_to_menu_button = Button(self, text="Perform", command=self.showStatsVisOutput)
        self.back_to_menu_button.pack(pady=5)

    def showStatsVisOutput(self):
        # Show the modal dialog
        response = ModalView.show_stat_vis_output()
        if response:
            print("Vis pos")
            # Navigate to the DataOutputView after file upload
            """Handle the 'Back' button."""
            current_view_name = [name for name, view in self.controller.houser.views.items() if view is self][0]
            print(f"{current_view_name}: Back button clicked!")
            #self.controller.houser.show_view("DataOutputView")
        else:
            print("Vis not pos!")
        print("StatsVisOutputsView")


if __name__ == "__main__":
    # Initialize the root view
    root = RootView()

    # Start the application
    root.mainloop()


2024-12-07 01:40:09.317 python[55389:2996391] Warning: Expected min height of view: (<NSButton: 0x7fa06f6b1320>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2024-12-07 01:40:09.318 python[55389:2996391] Warning: Expected min height of view: (<NSButton: 0x7fa06f69fd90>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


Alert response: True


2024-12-07 01:40:12.153 python[55389:2996391] Warning: Expected min height of view: (<NSButton: 0x7fa06f878000>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2024-12-07 01:40:12.155 python[55389:2996391] Warning: Expected min height of view: (<NSButton: 0x7fa06f8774d0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


Alert response: True
smell True


Version I 

In [ ]:
from tkinter import *
from tkinter import messagebox


class RootView(Tk):
    """Root view representing the main application window."""

    def __init__(self):
        super().__init__()
        self.title("Dynamic Views and Navigation")
        self.geometry("500x400")  # Main window size
        self.navigation_stack = []  # Stack to track navigation history

        # Create the Houser view as a child of RootView
        self.houser = HouserView(self)
        self.houser.pack(fill="both", expand=True, padx=10, pady=10)

        # Add predefined views dynamically to the Houser view
        self.houser.add_view("WelcomeView", WelcomeView, controller=self)
        self.houser.add_view("MenuView", MenuView, controller=self)
        self.houser.add_view("FileInputView", FileInputView, controller=self, width=500, height=400, bg="lightblue")
        self.houser.add_view("DataOutputView", DataOutputView, controller=self, width=500, height=400, bg="lightgreen")
        self.houser.add_view("StatsVisMethodsView", StatsVisMethodsView, controller=self, width=500, height=400, bg="lightyellow")

        # Show the initial view
        self.houser.show_view("WelcomeView")

    def update_title(self, new_title):
        """Update the title of the window."""
        self.title(new_title)


class HouserView(Frame):
    """Container for all dynamic views."""

    def __init__(self, parent, width=500, height=400, bg="yellow"):
        super().__init__(parent, width=width, height=height, bg=bg)
        self.parent = parent
        self.views = {}  # To track all dynamic views by name
        self.pack_propagate(False)

    def add_view(self, view_name, view_class, **kwargs):
        """
        Dynamically add a view to the Houser view.
        :param view_name: The name of the view (used as a key in the views dictionary).
        :param view_class: The class of the view to add.
        :param kwargs: Additional arguments for the view class.
        :return: The created view instance.
        """
        view = view_class(self, **kwargs)
        self.views[view_name] = view
        view.grid(row=0, column=0, sticky="nsew")  # Use grid to stack views
        return view

    def show_view(self, view_name):
        """
        Show the view with the given name.
        :param view_name: The name of the view to display.
        """
        if view_name in self.views:
            current_view = self.get_current_view_name()
            if current_view and current_view != view_name:
                # Add the current view to the navigation stack if it's not already the last entry
                if not self.parent.navigation_stack or self.parent.navigation_stack[-1] != current_view:
                    self.parent.navigation_stack.append(current_view)
            self.views[view_name].tkraise()

            # Map view names to friendly titles
            friendly_titles = {
                "WelcomeView": "Welcome Page",
                "MenuView": "Main Menu",
                "FileInputView": "File Upload Page",
                "DataOutputView": "Data Output Page",
                "StatsVisMethodsView": "Statistics/Visualisations Methods Page"
            }
            self.parent.update_title(friendly_titles.get(view_name, view_name))

    def get_current_view_name(self):
        """Get the name of the currently visible view."""
        for name, view in self.views.items():
            if view.winfo_ismapped():
                return name
        return None


class NavigationBar(Frame):
    """Generic Navigation Bar."""

    def __init__(self, parent, on_back_pressed, height=50, bg="gray"):
        super().__init__(parent, height=height, bg=bg)
        self.parent = parent
        self.pack_propagate(False)

        # Create the "Back" button
        self.back_button = Button(self, text="Back", command=on_back_pressed)
        self.back_button.pack(side="left", padx=10, pady=5)


class BaseView(Frame):
    """Base class for all views."""

    def __init__(self, parent, controller, **kwargs):
        super().__init__(parent, **kwargs)
        self.controller = controller

    def add_navigation_bar(self, custom_on_back_pressed=None):
        """Add a navigation bar with an optional custom back button handler."""
        on_back_pressed = custom_on_back_pressed or self.on_back_pressed
        self.navbar = NavigationBar(self, on_back_pressed)
        self.navbar.pack(fill="x", side="top")

    def on_back_pressed(self):
        """Handle the 'Back' button."""
        if self.controller.navigation_stack:
            print(f"No pres view{self.controller.navigation_stack}")
            # Pop the last view and navigate to it
            previous_view = self.controller.navigation_stack.pop()
            print(f"Prev view{previous_view}")
            self.controller.houser.show_view(previous_view)
        else:
            print("No previous view to navigate to!")


class AlertView:
    """Alert View to show an interactive dialog."""

    @staticmethod
    def show_error():
        response = messagebox.showerror("Error", "An error occurred!")
        print(f"Alert response: {response}")
        return response

    @staticmethod
    def show_query():
        response = messagebox.askyesno("Confirmation", "Proceed?")
        print(f"Alert response: {response}")
        return response


class WelcomeView(BaseView):
    """Predefined Welcome View."""

    def __init__(self, parent, controller):
        super().__init__(parent, controller)

        # Add a welcome label
        self.label = Label(self, text="Welcome to the Program!")
        self.label.pack(pady=5)

        # Add a button to navigate to the next view
        self.continue_button = Button(self, text="Continue", command=self.on_continue_clicked)
        self.continue_button.pack(pady=5)

    def on_continue_clicked(self):
        """Handle the 'Continue' button click."""
        response = AlertView.show_query()
        if response:
            self.controller.houser.show_view("MenuView")


class MenuView(BaseView):
    """Predefined Menu View."""

    def __init__(self, parent, controller):
        super().__init__(parent, controller)

        # Add a navigation bar
        self.add_navigation_bar()

        # Add a menu label
        self.label = Label(self, text="This is the Menu View.")
        self.label.pack(pady=5)

        # Add buttons to navigate to different views
        self.file_input_button = Button(self, text="Go to File Input", command=self.on_file_input_clicked)
        self.file_input_button.pack(pady=5)

        self.data_output_button = Button(self, text="Go to Data Output", command=self.on_data_output_clicked)
        self.data_output_button.pack(pady=5)

    def on_file_input_clicked(self):
        """Navigate to the File Input View."""
        self.controller.houser.show_view("FileInputView")

    def on_data_output_clicked(self):
        """Navigate to the Data Output View."""
        self.controller.houser.show_view("DataOutputView")


class FileInputView(BaseView):
    """Predefined File Input View."""

    def __init__(self, parent, controller, **kwargs):
        super().__init__(parent, controller, **kwargs)

        # Add a navigation bar
        self.add_navigation_bar()

        # Add a file input label
        self.label = Label(self, text="This is the File Input View.")
        self.label.pack(pady=5)


class DataOutputView(BaseView):
    """Data Output View."""

    def __init__(self, parent, controller, **kwargs):
        super().__init__(parent, controller, **kwargs)

        # Add a navigation bar
        self.add_navigation_bar()

        # Add a data output label
        self.label = Label(self, text="This is the Data Output View.")
        self.label.pack(pady=5)


class StatsVisMethodsView(BaseView):
    """Stats Visualisation Methods View."""

    def __init__(self, parent, controller, **kwargs):
        super().__init__(parent, controller, **kwargs)

        # Add a navigation bar
        self.add_navigation_bar()

        # Add a stats label
        self.label = Label(self, text="This is the Stats Visualisation Methods View.")
        self.label.pack(pady=5)


if __name__ == "__main__":
    # Initialize the root view
    root = RootView()

    # Start the application
    root.mainloop()


2024-12-07 17:41:24.882 python[55389:2996391] Warning: Expected min height of view: (<NSButton: 0x7fa06ee20f90>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2024-12-07 17:41:24.885 python[55389:2996391] Warning: Expected min height of view: (<NSButton: 0x7fa06ee0d150>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


Alert response: True
No pres view['WelcomeView']
Prev viewWelcomeView


Version J

- Element
  - UIElements
    - Views
      - HouserView
      - PageView
        - WelcomeView
        - MenuView
        - FileInputView
        - DataOutputView
        - Stats/VisMethodsView
      - AlertView
        - Error
        - Warning
        - Confirmation
      - ModalView
        - BrowseFiles
        - Edit/Update
        - DisplayOutput
        - Stats/Vis
    - Section
      - Header
      - Navigation
      - Body
      - Footer
    - UserInput
    - UserOutput
  - UXElements
    - Action
      - Transitions
        - goToNextPage
        - goToPreviousPage
    - Listener


Implementation is like:

define instances of "WelcomeView", "MenuView", "FileInputView" etc

create root element object

create houser view in root to house all subsequent views

display relevant PageView


Tiramisuper

In [29]:
import tkinter as tk
from tkinter import messagebox

class Element(tk.Frame):
    def __init__(self, master=None, name="Element"):
        super().__init__(master)
        self.name = name
        self.master = master

    def describe(self):
        if self.name is not None:
            return f"{self.name} is an instance of {self.__class__.__name__}"

class UIElements(Element):
    views = []  # To track all dynamic views by name

class UXElements(Element):
    pass

class Actions(UXElements):
    def __init__(self, container, name="Views"):
        super().__init__(container, name)
        self.container = container
        self.current_view = None
        print(self.describe())  # Fixed: Added parentheses

    def add_view(self, view_name, view_class, **kwargs):
        """
        Dynamically add a view to the Houser view.
        """
        self.current_view = view_class(self.container, **kwargs)
        UIElements.views.append(self.current_view)  # Fixed: Corrected append syntax
        print(f"Added view: {self.current_view}")
        self.current_view.grid(row=0, column=0, sticky="nsew")
        return self.current_view

    def load_view(self, view_class):
        """Load standard views or modal views."""
        
        if issubclass(view_class, ModalView):
            print(f"Modal view: {view_class.name}")
            modal_view = view_class(self.container)
            
            self.current_view = view_class(self.container)

            master = self.current_view.parent
            
            

            
            modal_view.transient(self)  # Set parent window
            modal_view.grab_set()

             # Center the modal window
            self.center_modal_window(modal_view, self)

            # Wait for the modal window
            self.wait_window(modal_view)

        elif issubclass(view_class, AlertView):
            print(f"Alert view: {view_class.name}")
            alert_view = view_class(self.container)
            
            self.current_view = view_class(self.container)
            
            

            
            alert_view.transient(self.master)  # Set parent window
            alert_view.grab_set()

             # Center the modal window
            self.center_modal_window(alert_view, self)

            # Wait for the modal window
            self.wait_window(alert_view)
        else:
            if view_class == FileInputView:
                print("The Gayest wabiski")
                self.current_view = self.container
                self.current_view.title("Wabiski")
            else:
                #if self.current_view:
                #    self.current_view.grid_remove()
                
                    # Instantiate the view if it's not already an instance
                if isinstance(view_class, type):  # If it's a class, instantiate it
                    self.current_view = view_class(self.container)
                    print("midgard")
                else:  # It's already an instance
                    self.current_view = view_class(self.container)
                    print("mudgard")

                self.current_view.grid(row=0, column=0, sticky="nsew")
                self.title(f"Curremps View: {self.current_view.name}")
            """     # Safeguard to ensure title is a callable
            if hasattr(self.current_view, "name"):
                print("midgard")
                #self.title()
                self.title(f"Curremps View: {self.current_view.name}")
            else:
                print("Error: master.title is not callable.")
            print(f"Loaded view: {self.current_view.name}") """

    def center_modal_window(self, window, parent_view):
        """Center the modal window relative to its parent."""
        window.update_idletasks()
        
        # Get parent window dimensions and position
        parent_x = parent_view.winfo_rootx()
        parent_y = parent_view.winfo_rooty()
        parent_width = parent_view.winfo_width()
        parent_height = parent_view.winfo_height()
        print(f"Position: {parent_x} by {parent_y}, Dimensions: {parent_width} by {parent_height}")
        
        # Get modal window dimensions
        window_width = window.winfo_width()
        window_height = window.winfo_height()
        print(f"Modal/Alert Dimensions: ({window_width}x{window_height})")
        
        # Calculate center position
        x = parent_x + (parent_width - window_width) // 2
        y = parent_y + (parent_height - window_height) // 2
        
        # Set modal window position
        window.geometry(f"{window_width}x{window_height}+{x}+{y}")
        print(f"Modal/Alert window {window.name} positioned at ({x}, {y}).")

    def nextMethodA(self):
        print("next_method_A")
        print(f"scrobe: {self.describe()}")  

    def prevMethodA(self):
        print("prev_method_A")
        print(f"screbe: {self.describe()}")  

    def duvies(self):
        print("fsiii")

    def duvpus(self):
        print("dsonna")

class Views(UIElements):
    def __init__(self, master=None, name="Views"):
        super().__init__(master, name)

class PageView(Views, Actions):
    def __init__(self, master=None, name="PageView"):
        super().__init__(master, name)
        Views.__init__(self, master, name)
        Actions.__init__(self, master, name)
        self.views = []  # Added: Initialize views list
        self.view_index = 0  # Added: Initialize view index
        

    def next_view(self):
        self.nextMethodA()
        if hasattr(self, 'views') and self.view_index < len(self.views) - 1:
            self.view_index += 1
            next_view_class = self.views[self.view_index]
            self.load_view(next_view_class)
        else:
            #super().next_view()
            #self.load_view(next_view_class)
            print("End of views.")
        


    def previous_view(self):
        self.prevMethodA()
        if hasattr(self, 'views') and self.view_index > 0:
            print("Thisgame")
            self.view_index -= 1  # Fixed: Simplified view index calculation
            self.load_view(self.views[self.view_index])
        else:
            print("Thugame")

class HouserView(PageView):
    def __init__(self, master=None):
        super().__init__(master, "HouserView")
        label = tk.Label(self, text="This is the Houser View")
        label.grid(row=0, column=0, pady=10)  # Changed from pack to grid

class MainWindow(tk.Tk, PageView):
    def __init__(self):
        super().__init__()
        self.name = "MainWindow"  # Add a name attribute
        self.title("Main Windowed")
        self.current_view = None
        self.geometry("600x400")

        self.views = [WelcomeView, MenuView, DataOutputView, Stat_VisMethodsView]
        self.view_index = 0
        
        # Navigation frame
        self.navigation_frame = tk.Frame(self, bg="lightgray")
        self.navigation_frame.grid(row=0, column=0, sticky="ew")  # Changed from pack to grid
        
        self.back_button = tk.Button(self.navigation_frame, text="Back", command=self.previous_view)
        self.back_button.grid(row=0, column=0, padx=5, pady=5)  # Changed from pack to grid
        
        self.next_button = tk.Button(self, text="Next View", command=self.next_view)
        self.next_button.grid(row=2, column=0, padx=5, pady=5)  # Changed from pack to grid

        # Create container for views
        self.container = tk.Frame(self)
        self.container.grid(row=1, column=0, sticky="nsew")  # Changed from pack to grid
        
        # Configure grid weights
        self.grid_rowconfigure(1, weight=1)
        self.grid_columnconfigure(0, weight=1)
        
        # Create the Houser view
        self.houser = HouserView(self.container)
        self.houser.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)  # Changed from pack to grid

        # Initialize views
        self.current_view = self.views[0](self.container)
        self.current_view.grid(row=0, column=0, sticky="nsew")
        #Actions.update_title(self, new_title=self.current_view.name)  # Update title for the initial view
        #self.update_title(f"funkyyy: {self.current_view.name}")  # Update title for the initial view

class WelcomeView(PageView):
    def __init__(self, master=None):
        super().__init__(master, "WelcomeView")
        label = tk.Label(self, text="Welcome to the application!")
        label.grid(row=0, column=0, pady=10)  # Changed from pack to grid

        btn1 = tk.Button(self, text="Alert: Error", command=self.duvies)
        btn2 = tk.Button(self, text="Alert: Warning", command=self.duvpus)
        btn3 = tk.Button(self, text="Alert: Confirmation", command=self.duvpus)
        btn1.grid(row=1, column=0, pady=5)  # Changed from pack to grid
        btn2.grid(row=2, column=0, pady=5)  # Changed from pack to grid
        btn3.grid(row=3, column=0, pady=5)  # Changed from pack to grid
    name = "WelcomeView"  # Add a name attribute

class MenuView(PageView):
    name = "MenuView"  # Add a name attribute
    title = "I'm Gay"
    def __init__(self, master=None, title=None):
        title = "I'm Gay"
        super().__init__(master, "MenuView")
        label = tk.Label(self, text="Menu")
        label.grid(row=0, column=0, pady=5)  # Changed from pack to grid
        
        btn1 = tk.Button(self, text="Alert: Error", command=lambda: self.load_view(Error))
        btn2 = tk.Button(self, text="Alert: Warning", command=lambda: self.load_view(Warning))
        btn3 = tk.Button(self, text="Alert: Confirmation", command=lambda: self.load_view(Confirmation))
        btn1.grid(row=1, column=0, pady=5)  # Changed from pack to grid
        btn2.grid(row=2, column=0, pady=5)  # Changed from pack to grid
        btn3.grid(row=3, column=0, pady=5)  # Changed from pack to grid
        
        #FIbtn = tk.Button(self, text="File Input", command=lambda: print("sak"))
        #FIbtn = tk.Button(self, text="File Input", command=lambda: print("sak"))
        FIbtn = tk.Button(self, text="File Input", command=lambda: self.load_view(FileInputView))
        #btn2 = tk.Button(self, text="Option 2", command=lambda: self.load_view(FileInputView))
        FIbtn.grid(row=15, column=0, pady=5)  # Changed from pack to grid
       # btn2.grid(row=2, column=0, pady=5)  # Changed from pack to grid
    

class FileInputView(PageView):
    view_title = "I'm Gayer"
    name = "FileInputView"  # Add a name attribute
    def __init__(self, master=None):
        super().__init__(master, "FileInputView")
        #self.controller = controller  # Store the controller reference
        
        labelA = tk.Label(self, text="Upload a file:")
        labelA.grid(row=0, column=0, pady=5)  # Changed from pack to grid
        #browseFiles = BrowseFiles()
        browseBtn = tk.Button(self, text="Browse Files", command=lambda: self.load_view(BrowseFiles))
        browseBtn.grid(row=1, column=0, pady=5)  # Changed from pack to grid


        labelB = tk.Label(self, text="Edit/Update a file:")
        labelB.grid(row=4, column=0, pady=5)  # Changed from pack to grid
        #browseFiles = BrowseFiles()
        editBtn = tk.Button(self, text="Update Files", command=lambda: self.load_view(EditUpdate))
        editBtn.grid(row=5, column=0, pady=5)  # Changed from pack to grid

        btn1 = tk.Button(self, text="Alert: Error", command=lambda: self.load_view(Error))
        btn2 = tk.Button(self, text="Alert: Warning", command=lambda: self.load_view(Warning))
        btn3 = tk.Button(self, text="Alert: Confirmation", command=lambda: self.load_view(Confirmation))
        btn1.grid(row=8, column=0, pady=5)  # Changed from pack to grid
        btn2.grid(row=9, column=0, pady=5)  # Changed from pack to grid
        btn3.grid(row=20, column=0, pady=5)  # Changed from pack to grid
    


    #PageView.next_view(self) = print(f"Custom next_view invoked from FileInputView: {PageView.name}")

    

class DataOutputView(PageView):
    def __init__(self, master=None):
        super().__init__(master, "DataOutputView")
        label = tk.Label(self, text="Data Output")
        label.grid(row=0, column=0, pady=5)  # Changed from pack to grid
        
        text = tk.Text(self, height=10, width=30)
        text.grid(row=1, column=0, pady=5)  # Changed from pack to grid
        btn1 = tk.Button(self, text="Alert: Error", command=lambda: self.load_view(Error))
        btn2 = tk.Button(self, text="Alert: Warning", command=lambda: self.load_view(Warning))
        btn3 = tk.Button(self, text="Alert: Confirmation", command=lambda: self.load_view(Confirmation))
        btn1.grid(row=1, column=0, pady=5)  # Changed from pack to grid
        btn2.grid(row=2, column=0, pady=5)  # Changed from pack to grid
        btn3.grid(row=3, column=0, pady=5)  # Changed from pack to grid
    name = "DataOutputView"  # Add a name attribute

class Stat_VisMethodsView(PageView):
    def __init__(self, master=None):
        super().__init__(master, "Stat_VisMethodsView")
        label = tk.Label(self, text="Stat_VisMethods")
        label.grid(row=0, column=0, pady=5)  # Changed from pack to grid
        
        text = tk.Text(self, height=10, width=30)
        text.grid(row=1, column=0, pady=5)  # Changed from pack to grid

        btn1 = tk.Button(self, text="Alert: Error", command=lambda: self.load_view(Error))
        btn2 = tk.Button(self, text="Alert: Warning", command=lambda: self.load_view(Warning))
        btn3 = tk.Button(self, text="Alert: Confirmation", command=lambda: self.load_view(Confirmation))
        btn1.grid(row=1, column=0, pady=5)  # Changed from pack to grid
        btn2.grid(row=2, column=0, pady=5)  # Changed from pack to grid
        btn3.grid(row=3, column=0, pady=5)  # Changed from pack to grid
    name = "Stat_VisMethodsView"  # Add a name attribute
    

class AlertView(tk.Toplevel):
    pass
    def __init__(self, master=None, name="AlertView"):
        super().__init__(master)

class Error(AlertView):
    name = "AlertView_Error"
    def __init__(self, master=None):
        super().__init__(master, "Error")
        #messagebox.showerror("Error", "An error has occurred!")
        print("longibbibibbies")
        self.title("Alert")
        label = tk.Label(self, text="Alert Errors")
        label.pack(pady=10)
        close_button = tk.Button(self, text="Close", command=self.destroy)
        close_button.pack(pady=10)

class Warning(AlertView):
    name = "AlertView_Warning"
    def __init__(self, master=None):
        super().__init__(master, "Warning")
        #messagebox.showwarning("Warning", "This is a warning!")
        self.title("Alert Warmomg")
        label = tk.Label(self, text="Alert Warnings")
        label.pack(pady=10)
        close_button = tk.Button(self, text="Close", command=self.destroy)
        close_button.pack(pady=10)

class Confirmation(AlertView):
    name = "AlertView_Confirmation"
    def __init__(self, master=None):
        super().__init__(master, "Confirmation")
        #messagebox.showinfo("Confirmation", "Action confirmed!")
        self.title("Alert Confirmation")
        label = tk.Label(self, text="Alert Confirmations")
        label.pack(pady=10)
        close_button = tk.Button(self, text="Close", command=self.destroy)
        close_button.pack(pady=10)

class ModalView(tk.Toplevel):
    pass
    def __init__(self, master=None, name="ModalView"):
        super().__init__(master)
        #self.title(name)
        #self.transient(master)
        #self.grab_set()
        #self.name = self.__class__.__name__
        # The "return value" of the dialog,
        # entered by user in self.entry Entry box.
        #self.data = None
        #tk.Toplevel.__init__()

        """ self.root=Toplevel(master)
        self.entry = Entry(self.root)
        self.entry.pack() """
        
        # Configure modal window
        #self.title(self.name)  # Set the window title
       # self.resizable(False, False)  # Optional: prevent resizing

        # Make the modal window grab focus and block interaction with other windows
        #self.transient(master)
       # self.grab_set()

class BrowseFiles(ModalView):
    name = "BrowseFiles"  # Add a name attribute
    def __init__(self, master=None):
        #super().__init__(master, "BrowseFiles")
        #label = tk.Label(self, text="Browse for files")
        #label.pack(pady=10)
        print("Thugadddme")
        super().__init__(master)
        self.title("Browse Files")
        label = tk.Label(self, text="Browse for files")
        label.pack(pady=10)

        btn1 = tk.Button(self, text="Alert: Error", command=lambda: self.load_view(Error))
        btn2 = tk.Button(self, text="Alert: Warning", command=lambda: self.load_view(Warning))
        btn3 = tk.Button(self, text="Alert: Confirmation", command=lambda: self.load_view(Confirmation))
        btn1.grid(row=1, column=0, pady=5)  # Changed from pack to grid
        btn2.grid(row=2, column=0, pady=5)  # Changed from pack to grid
        btn3.grid(row=3, column=0, pady=5)  # Changed from pack to grid
        close_button = tk.Button(self, text="Close", command=self.destroy)
        close_button.pack(pady=10)

class EditUpdate(ModalView):
    name = "Edit/Update"
    def __init__(self, master=None):
        super().__init__(master)
        self.title("Edit/Update")
        label = tk.Label(self, text="Edit or Update")
        label.pack(pady=10)

        btn1 = tk.Button(self, text="Alert: Error", command=lambda: self.load_view(Error))
        btn2 = tk.Button(self, text="Alert: Warning", command=lambda: self.load_view(Warning))
        btn3 = tk.Button(self, text="Alert: Confirmation", command=lambda: self.load_view(Confirmation))
        btn1.grid(row=1, column=0, pady=5)  # Changed from pack to grid
        btn2.grid(row=2, column=0, pady=5)  # Changed from pack to grid
        btn3.grid(row=3, column=0, pady=5)  # Changed from pack to grid
        close_button = tk.Button(self, text="Close", command=self.destroy)
        close_button.pack(pady=10)

class DisplayOutput(ModalView):
    name = "Display Output"  # Add a name attribute
    def __init__(self, master=None):
        super().__init__(master)
        self.title("Output Display")
        label = tk.Label(self, text="Displaying output")
        label.pack(pady=10)
        
        btn1 = tk.Button(self, text="Alert: Error", command=lambda: self.load_view(Error))
        btn2 = tk.Button(self, text="Alert: Warning", command=lambda: self.load_view(Warning))
        btn3 = tk.Button(self, text="Alert: Confirmation", command=lambda: self.load_view(Confirmation))
        btn1.grid(row=1, column=0, pady=5)  # Changed from pack to grid
        btn2.grid(row=2, column=0, pady=5)  # Changed from pack to grid
        btn3.grid(row=3, column=0, pady=5)  # Changed from pack to grid
        close_button = tk.Button(self, text="Close", command=self.destroy)
        close_button.pack(pady=10)

if __name__ == "__main__":
    app = MainWindow()
    app.mainloop()



HouserView is an instance of HouserView
HouserView is an instance of HouserView
HouserView is an instance of HouserView
WelcomeView is an instance of WelcomeView
WelcomeView is an instance of WelcomeView
WelcomeView is an instance of WelcomeView
next_method_A
scrobe: MainWindow is an instance of MainWindow
MenuView is an instance of MenuView
MenuView is an instance of MenuView
MenuView is an instance of MenuView
midgard
The Gayest wabiski


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/tkinter/__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/21/g6828w8x4ys2wkm155kcs8rh0000gn/T/ipykernel_48156/4061851894.py", line 255, in <lambda>
    FIbtn = tk.Button(self, text="File Input", command=lambda: self.load_view(FileInputView))
                                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/21/g6828w8x4ys2wkm155kcs8rh0000gn/T/ipykernel_48156/4061851894.py", line 81, in load_view
    self.current_view.title("Wabiski")
    ^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Frame' object has no attribute 'title'


KeyboardInterrupt: 

Class Heirarchy 

For implementation of real world modelling of jam jars

Using Rapid Prototyping and Agile development 

Create a programmatic version of a real world object. 
Initial implementation defines only rudimentary functionality (like predifine all views, load first view, test app flow)

Program looks to model real world object/method/algorithm through dynamic descriptions of object: attributes and methods.

["Jam Jar"]
class object:
    name,
    width,
    height

class widget(object):
    
class event(object):

class listener(object):

class shape(widget):
    type, (2dimensional or 3dimensional)
    case, (yes or no (yes means must be a container or parent, no means must be a child))
    
class fruit(object)

class strawberry(fruit):
    #logic to show strawberry as fruit

class orange(fruit):
    #logic to show orange as fruit 

class jam(object)
    #logic to show flavour takes from intance of fruit obj (strawb/orange)

class jar(shape)

In [37]:
class Object:
    def __init__(self, name, width, height):
        self.name = name
        self.width = width
        self.height = height

    def describe(self):
        return f"{self.name}: Width={self.width}, Height={self.height}"

class Widget(Object):
    def __init__(self, name, width, height):
        super().__init__(name, width, height)
        self.events = []

    def add_event(self, event):
        self.events.append(event)

    def handle_events(self):
        for event in self.events:
            event.trigger()

class Event(Object):
    def __init__(self, name, callback):
        super().__init__(name, 0, 0)
        self.callback = callback

    def trigger(self):
        print(f"Event {self.name} triggered.")
        self.callback()

class Listener(Object):
    def __init__(self, name, width, height):
        super().__init__(name, width, height)

    def listen(self, event):
        print(f"Listener {self.name} responding to {event.name}")

class Shape(Widget):
    def __init__(self, name, width, height, shape_type, case):
        super().__init__(name, width, height)
        self.shape_type = shape_type
        self.case = case

    def describe(self):
        base_description = super().describe()
        return f"{base_description}, Type={self.shape_type}, Case={self.case}"

class Fruit(Object):
    def __init__(self, name, width, height):
        super().__init__(name, width, height)

    def flavor(self):
        raise NotImplementedError("Subclasses must implement flavor method.")

class Strawberry(Fruit):
    def flavor(self):
        return "Sweet and tangy"

class Orange(Fruit):
    def flavor(self):
        return "Citrusy and tangy"

class Jam(Object):
    def __init__(self, name, width, height, fruit):
        super().__init__(name, width, height)
        if not isinstance(fruit, Fruit):
            raise TypeError("Jam must be made from a Fruit instance.")
        self.fruit = fruit

    def describe(self):
        base_description = super().describe()
        return f"{base_description}, Made from: {self.fruit.name}, Flavor: {self.fruit.flavor()}"

class Jar(Shape):
    def __init__(self, name, width, height):
        super().__init__(name, width, height, "3dimensional", True)
        self.contents = []

    def add_content(self, item):
        if isinstance(item, Jam):
            self.contents.append(item)
            print(f"Added {item.name} to {self.name}.")
        else:
            raise TypeError("Jar can only contain Jam instances.")

    def list_contents(self):
        if not self.contents:
            return "Jar is empty."
        return "Contents: " + ", ".join(item.name for item in self.contents)

# Example Usage:
if __name__ == "__main__":
    strawberry = Strawberry("Strawberry", 1, 1)
    orange = Orange("Orange", 1, 1)

    strawberry_jam = Jam("Strawberry Jam", 5, 5, strawberry)
    orange_jam = Jam("Orange Jam", 5, 5, orange)

    jar = Jar("Jam Jar", 10, 15)
    print(jar.describe())

    jar.add_content(strawberry_jam)
    jar.add_content(orange_jam)

    print(jar.list_contents())


Jam Jar: Width=10, Height=15, Type=3dimensional, Case=True
Added Strawberry Jam to Jam Jar.
Added Orange Jam to Jam Jar.
Contents: Strawberry Jam, Orange Jam


In [48]:
class Person:
  def __init__(self, fname, lname):
    self.firstname = fname
    self.lastname = lname

  def printname(self):
    print(self.firstname, self.lastname)

class Student(Person):
  def __init__(self, fname, lname, year):
    super().__init__(fname, lname)
    self.graduationyear = year

x = Student("Mike", "Olsen", 2019)
print(x.graduationyear)



2019


In [17]:
import tkinter as tk

# Base class for all frames (visual elements or containers)
class Frame(tk.Frame):
    def __init__(self, parent, logical_name):
        super().__init__(parent)
        self.logical_name = logical_name
        self.parent = parent
        self.my_children = []

    def add_child(self, child):
        self.my_children.append(child)
        child.pack(fill="both", expand=True)  # Automatically pack child frames
        print(f"{child.logical_name} added to {self.logical_name}")

    def announce(self):
        if self.parent and hasattr(self.parent, "logical_name"):
            print(f"My name is {self.logical_name} and I am a child of {self.parent.logical_name}")
        else:
            print(f"My name is {self.logical_name} and I have no parent")

        # Recursively announce for all children
        for child in self.my_children:
            if isinstance(child, Frame):
                child.announce()

# Top-level App Frame
class App(Frame):
    def __init__(self, root):
        super().__init__(root, "App")
        self.pack(fill="both", expand=True)
        self.current_view = None  # Track the currently visible view

    def show_view(self, view):
        """Show the given view and hide the current one."""
        if self.current_view:
            self.current_view.pack_forget()  # Hide the current view
        view.pack(fill="both", expand=True)  # Show the new view
        self.current_view = view  # Update the current view

# Subframe: Houser
class Houser(Frame):
    def __init__(self, parent):
        super().__init__(parent, "Houser")
        self.config(bg="lightblue")

# Subframe: WelcomeView
class WelcomeView(Frame):
    def __init__(self, parent, app):
        super().__init__(parent, "WelcomeView")
        self.config(bg="lightgreen")

        # Add a button to proceed to MenuView
        button = tk.Button(self, text="Proceed to Menu", command=lambda: app.show_view(parent.menu_view))
        button.pack(pady=10)

# Subframe: MenuView
class MenuView(Frame):
    def __init__(self, parent, app):
        super().__init__(parent, "MenuView")
        self.config(bg="lightyellow")

        # Add a button to proceed to the final view or perform actions
        button_next = tk.Button(self, text="Proceed to Final View", command=lambda: app.show_view(parent.final_view))
        button_next.pack(pady=10)

# Subframe: FinalView
class FinalView(Frame):
    def __init__(self, parent):
        super().__init__(parent, "FinalView")
        self.config(bg="lightcoral")

        # Add a label to indicate completion
        label = tk.Label(self, text="You have reached the end!", font=("Arial", 16))
        label.pack(pady=20)

# Initialize the application
root = tk.Tk()
root.title("Sequential View App with tkinter")
root.geometry("400x300")  # Set the initial size of the app window

# Initialize the structure
app = App(root)
houser = Houser(app)

# Store references to the views in the Houser frame
houser.welcome_view = WelcomeView(houser, app)
houser.menu_view = MenuView(houser, app)
houser.final_view = FinalView(houser)

# Add subframes to Houser
houser.add_child(houser.welcome_view)
houser.add_child(houser.menu_view)
houser.add_child(houser.final_view)

# Add Houser to the App and start with WelcomeView
app.add_child(houser)
app.show_view(houser.welcome_view)

# Display the hierarchy
print("\n--- Automatic Hierarchy Announcement ---")
app.announce()

# Start the tkinter event loop
root.mainloop()


WelcomeView added to Houser
MenuView added to Houser
FinalView added to Houser
Houser added to App

--- Automatic Hierarchy Announcement ---
My name is App and I have no parent
My name is Houser and I am a child of App
My name is WelcomeView and I am a child of Houser
My name is MenuView and I am a child of Houser
My name is FinalView and I am a child of Houser


KeyboardInterrupt: 

In [30]:
class A:
    def __init__(self):
        print("Initializing A")

class B(A):
    def __init__(self):
        #super().__init__()
        print("Initializing B")
        
    def kudge():
        print("Initializing Kudge")

class C(B):
    def __init__(self):
        A.__init__(self)  # Directly call A's __init__ without invoking B or C
        B.__init__(self)
        B.kudge()
        # Skips B's initialization and avoids "Initializing B" and "Initializing C"

c = C()

Initializing A
Initializing B
Initializing Kudge


Tiramisuck

In [20]:
import tkinter as tk
#from tkinter import ttk
from tkinter import filedialog
#CURR_OBJECTTIVE: Centraalise functions, in Actions class. working with self.local_authorities
# Base class for all views
class Window(tk.Frame):
    def __init__(self, master, controller):
        super().__init__(master)
        self.controller = controller
        #Ehi create navigation pane
       # self.create_navigation_buttons()

    def localAuthorities(self):
        print("Local Auth")
        #make an action and assign to button

    def create_navigation_buttons(self):
        # Back button
        self.back_button = tk.Button(self, text="Back", command=self.controller.go_back)
        self.back_button.pack(side="left", padx=10, pady=10)

        # Next button
        self.next_button = tk.Button(self, text="sandboxNext", command=self.controller.go_next)
        self.next_button.pack(side="right", padx=10, pady=10)

class Actions:
    def __init__(self, name=None, master=None):
        self.name = name
        self.master = master


    def uploadFile(self):
        print(f"(Generic action to save file to drive {self.name})")

    def add_view(self):
        print("add views func")

    def show_view(self, view_class):
        print(f"(Prev view {self.master.__class__})")
        print(f"(Generic action to show view {self.name})")
        """ if self.current_view is not None:
            self.current_view.pack_forget()
        self.current_view = view_class(self, self)
        self.stack.append(self.current_view)
        self.current_view.pack(fill="both", expand=True) """
        if self.master.current_view is None:
            print("osgay")
        else:
            self.master.current_view.pack_forget()
        self.master.current_view = view_class(self.master, self)
        # Instantiate the new view
        
        
        # Append to the view stack (optional, if managing history)
        #self.stack.append(self.current_view)
        
        # Pack the new view into the main window
        self.master.current_view.pack(fill="both", expand=True)
        if view_class == WelcomeView:
            #self.current_view = view_class
            print(f"(View Worked {self.name})")
            #self.stack.append(self.current_view)
            #self.current_view = view_class(self, self)
            #WelcomeView.pack(self.current_view, fill="both", expand=True)

class displayFrame(Window):
    #super().__init__(master)
    print()

class View(displayFrame):
    def __init__(self, master=None, name="PageView"):
        super().__init__(master, name)
        #self.views = []  # Added: Initialize views list
        #self.view_index = 0  # Added: Initialize view index
        # 

    def file_selected():
        print("gold")

    def open_file_browser(self):
        ModalView(self, self.file_selected)     





#Navigation
class NavigationFrame(displayFrame):

    def __init__(self):
        #super().localAuthorities()
        print("navi")
        self.stack = []  # Stack for navigation

# AlertView for handling errors
class AlertView(tk.Toplevel):
    def __init__(self, message):
        super().__init__()
        self.title("Alert")
        label = tk.Label(self, text=message, foreground="red")
        label.pack(padx=20, pady=20)
        close_button = tk.Button(self, text="Close", command=self.destroy)
        close_button.pack(pady=10)

# ModalView for file browsing
class ModalView(tk.Toplevel):
    def __init__(self, master, callback):
        super().__init__(master)
        self.title("Browse Files")
        self.callback = callback
        tk.Label(self, text="Choose a file:").pack(padx=20, pady=10)
        self.file_button = tk.Button(self, text="Browse", command=self.browse_file)
        self.file_button.pack(padx=10, pady=10)

    def browse_file(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.callback(file_path)
            self.destroy()





# WelcomeView
class WelcomeView(View):
    def __init__(self, master, controller):
        super().__init__(master, controller)
        init_actions = Actions("WelcomeView", self)
        
        tk.Label(self, text="Welcome to the Tea App", font=("Arial", 16)).pack(pady=20)
        tk.Button(self, text="Go to Menu", command=lambda: init_actions.show_view(MenuView)).pack(pady=10)
        
# MenuView
class MenuView(View):
    def __init__(self, master, controller):
        super().__init__(master, controller)
        init_actions = Actions("MenuView", self)
        tk.Label(self, text="Menu", font=("Arial", 16)).pack(pady=20)
        
        tk.Button(self, text="File Input", command=lambda: init_actions.show_view(FileInputView)).pack(pady=10)
        tk.Button(self, text="Data Output", command=lambda: init_actions.show_view(DataOutputView)).pack(pady=10)

# FileInputView
class FileInputView(View):
    def __init__(self, master, controller):
        super().__init__(master, controller)
        
        
        self.localAuthorities()#MOEST RECENT- Trying to call localAuthorities(self)
        #^ This worked and printed "Locaal auth" " thru line 13
        tk.Label(self, text="File Input", font=("Arial", 16)).pack(pady=20)
        tk.Button(self, text="Open File Browser", command=self.open_file_browser).pack(pady=10)
        action = Actions("Party")
        action.uploadFile()  # Output: (easy Party)
    

    def file_selected(self, file_path):
        tk.Label(self, text=f"Selected File: {file_path}").pack(pady=10)
        self.controller.show_view(DataOutputView)  # Navigate to DataOutputView after file selection
        


# DataOutputView
class DataOutputView(View):
    def __init__(self, master, controller):
        super().__init__(master, controller)
        tk.Label(self, text="Data Output", font=("Arial", 16)).pack(pady=20)
        tk.Label(self, text="Here is your processed data.").pack(pady=10)
        tk.Button(self, text="Open File Browser", command=self.open_file_browser).pack(pady=10)

# Main Application Controller
class App(tk.Tk):
    """ def __new__(self):
        print("new") """
    def __init__(self, webdy, current_view):
        super().__init__()
        self.title("Tea App")
        self.geometry("600x400")
        #self.stack = []  # Stack for navigation
        #self.slen = len(self.stack)
        self.yani = webdy
        self.current_view = None
        init_actions = Actions("Cirrus", self)
        #init_actions.uploadFile()  # Output: (easy Party)
        # pseudocode- show first view
        #self.show_view(WelcomeView)
        init_actions.show_view(WelcomeView)

    
    


    """  def go_back(self):
        
        if self.slen > 1:
            self.current_view.pack_forget()
            self.current_view = self.stack.pop()
            self.current_view.pack(fill="both", expand=True)
        else:
            AlertView("No previous view to navigate to.")

    def go_next(self):
        if self.slen > 1:
            next_view = self.stack.pop(0)  # Pop from the beginning of the stack
            self.current_view.pack_forget()
            self.current_view = next_view
            self.current_view.pack(fill="both", expand=True)
        else:
            AlertView("No next view to navigate to.") """

    


if __name__ == "__main__":
    app = App(webdy="sosox", current_view=WelcomeView)
    app.mainloop()



(Prev view <class '__main__.App'>)
(Generic action to show view Cirrus)
osgay
(View Worked Cirrus)
(Prev view <class '__main__.WelcomeView'>)
(Generic action to show view WelcomeView)


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/tkinter/__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/21/g6828w8x4ys2wkm155kcs8rh0000gn/T/ipykernel_738/2638599861.py", line 132, in <lambda>
    tk.Button(self, text="Go to Menu", command=lambda: init_actions.show_view(MenuView)).pack(pady=10)
                                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/21/g6828w8x4ys2wkm155kcs8rh0000gn/T/ipykernel_738/2638599861.py", line 46, in show_view
    if self.master.current_view is None:
       ^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'WelcomeView' object has no attribute 'current_view'


In [21]:
# Import Module
from tkinter import *

# Create Tkinter Object
MainWindow = Tk()

# Set Geometry
MainWindow.geometry("848x480")

# Configure MainWindow grid to allow widgets to expand
MainWindow.grid_rowconfigure(0, weight=1)
MainWindow.grid_columnconfigure(0, weight=1)


class Window(Frame):
    def __init__(self, master=None):
        super().__init__(master)
        print("Window level initialized")


class Action:
    print("Actions initialized")


class Holder(Window):  # Holds all views like double bagging
    def __init__(self, master=None):
        super().__init__(master)
        print("Hold all views")


class View(Window):
    def __init__(self, master=None):
        super().__init__(master)
        print("Views initialized")


# Views
class WelcomeView(View):
    def __init__(self, master=None):
        super().__init__(master)
        print("This is the WelcomeView")

        # Creating the welcome_view as a LabelFrame
        self.welcome_view = LabelFrame(master, bg="blue", width=500, height=300)
        self.welcome_view.grid(row=0, column=0, sticky='nswe')

        # Adding a button to the welcome_view
        self.b1 = Button(self.welcome_view, text="Apple", command=self.on_button_click)
        self.b1.pack()

    def on_button_click(self):
        print("Apple button clicked")


""" class MenuView(View):
    #def __init__(self, master, controller):
        #self.master = master
    #super().__init__(MainWindow)
    print("This is the menu view")
        #print(f"master is: {self.master}")
    # menu_view
    menu_view = MenuView(MainWindow, bg="white",width=100,height=100)
    #menu_view.pack(pady=20,padx=20) """

    

 
# Adding the WelcomeView to the main window
welcome = WelcomeView(MainWindow)

# Execute Tkinter
MainWindow.mainloop()

Actions initialized
Window level initialized
Views initialized
This is the WelcomeView


In [22]:
from tkinter import *


# Create Tkinter Object
MainWindow = Tk()

# Set Geometry
MainWindow.geometry("848x480")

# Configure MainWindow grid to allow widgets to expand
MainWindow.grid_rowconfigure(0, weight=1)
MainWindow.grid_columnconfigure(0, weight=1)


class Window(Frame):
    def __init__(self, master=None):
        super().__init__(master)
        print("Window level initialized")


class Action:
    print("Actions initialized")


class Holder(Window):  # Holds all views like double bagging
    def __init__(self, master=None):
        super().__init__(master)
        print("Hold all views")


class View(Window):
    def __init__(self, master=None):
        super().__init__(master)
        print("Views initialized")


# WelcomeView
class WelcomeView(View):
    def __init__(self, master=None, controller=None):
        super().__init__(master)
        self.controller = controller
        print("This is the WelcomeView")

        # Creating the welcome_view as a LabelFrame
        self.welcome_view = LabelFrame(master, bg="blue", width=500, height=300)
        self.welcome_view.grid(row=0, column=0, sticky='nswe')

        # Adding a button to the welcome_view
        self.b1 = Button(self.welcome_view, text="Apple", command=self.on_button_click)
        self.b1.pack()

    def on_button_click(self):
        print("Apple button clicked")
        self.controller.show_menu_view()


# MenuView
class MenuView(View):
    def __init__(self, master=None, controller=None):
        super().__init__(master)
        self.controller = controller
        print("This is the MenuView")

        # Creating the menu_view as a LabelFrame
        self.menu_view = LabelFrame(master, bg="white", width=500, height=300)
        self.menu_view.grid(row=0, column=0, sticky='nswe')

        # Adding a button to go back to WelcomeView
        self.b2 = Button(self.menu_view, text="Back", command=self.on_button_click)
        self.b2.pack()

    def on_button_click(self):
        print("Back button clicked")
        self.controller.show_welcome_view()


# App Controller to manage views
class AppController:
    def __init__(self, root):
        self.root = root
        self.current_view = None
        self.show_welcome_view()

    def clear_view(self):
        """Clear the current view."""
        if self.current_view is not None:
            self.current_view.destroy()

    def show_welcome_view(self):
        """Show the WelcomeView."""
        self.clear_view()
        self.current_view = WelcomeView(self.root, controller=self)

    def show_menu_view(self):
        """Show the MenuView."""
        self.clear_view()
        self.current_view = MenuView(self.root, controller=self)


# Start the application
if __name__ == "__main__":
    app = AppController(MainWindow)
    MainWindow.mainloop()


Actions initialized
Window level initialized
Views initialized
This is the WelcomeView


In [23]:
# Import the library tkinter 
from tkinter import *

# Create a GUI app 
app = Tk() 

# Give a title to your app 
app.title("Vinayak App") 

# Constructing the first frame, frame1 
frame1 = LabelFrame(app, text="Fruit", bg="green", 
					fg="white", padx=15, pady=15) 

# Displaying the frame1 in row 0 and column 0 
frame1.grid(row=0, column=0) 

# Constructing the button b1 in frame1 
b1 = Button(frame1, text="Apple") 

# Displaying the button b1 
b1.pack() 

# Constructing the second frame, frame2 
frame2 = LabelFrame(app, text="Vegetable", bg="yellow", padx=15, pady=15) 

# Displaying the frame2 in row 0 and column 1 
frame2.grid(row=0, column=1) 

# Constructing the button in frame2 
b2 = Button(frame2, text="Tomato") 

# Displaying the button b2 
b2.pack() 

# Make the loop for displaying app 
app.mainloop() 


StackOF god

In [2]:
import tkinter as tk
from tkinter import ttk

LARGE_FONT = ("ariel", 20) # dont know what you had here

class Main(tk.Tk):
    def __init__(self, *args, **kwargs):

        tk.Tk.__init__(self, *args, **kwargs)
        self.grid_rowconfigure(0, weight=1) # this needed to be added
        self.grid_columnconfigure(0, weight=1) # as did this

        main_container = tk.Frame(self)
        main_container.grid(column=0, row=0, sticky = "nsew")
        main_container.grid_rowconfigure(0, weight = 1)
        main_container.grid_columnconfigure(0, weight = 1)

        menu_bar = tk.Menu(main_container)
        file_menu = tk.Menu(menu_bar, tearoff = 0) 
        file_menu.add_command(label = "Save settings", command = lambda: print("popupmsg('Not supported yet!')"))
        file_menu.add_separator()
        file_menu.add_command(label = "Exit", command = quit)
        menu_bar.add_cascade(label = "File", menu = file_menu)

        tk.Tk.config(self, menu = menu_bar)

        self.frames = {}

        for fr in (MainPage,):
            frame = fr(main_container, self)
            self.frames[fr] = frame
            frame.grid(row = 0, column = 0, sticky = "nsew")
        self.show_frame(MainPage)

    def show_frame(self, pointer):
        frame = self.frames[pointer]
        frame.tkraise()

class MainPage(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)

        # uncommented these lines
        self.columnconfigure(0, weight = 1)
        self.rowconfigure(0, weight = 1)
        self.rowconfigure(1, weight = 1)
        self.rowconfigure(2, weight = 1)
        self.rowconfigure(3, weight = 1)

        label = tk.Label(self, text = "Main Page", font = LARGE_FONT)
        label.grid(row = 0, padx = 10, pady = 10)

        button1 = ttk.Button(self, text = "Graphs", command = lambda: print("controller.show_frame(GraphsPage)"))
        button1.grid(row = 1, sticky = 'nswe')

        button2 = ttk.Button(self, text = "Page 2", command = lambda: print("controller.show_frame(Page2)"))
        button2.grid(row = 2, sticky = 'nswe')

        button3 = ttk.Button(self, text = "Exit", command = quit)
        button3.grid(row = 3, sticky = 'nswe')

app = Main()
app.geometry("1280x720")
app.mainloop()

In [24]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog

class DoTheThing:
    def __init__(self, name):
        self.message = "saddeg" + name

    def printmessage(self):
        print(f"londoon {self.message}")  

    def sendnudes(self):
        print(f"sendnudes and budes")

class ShowTheThing(tk.Tk):
    def __init__(self, name):
        super().__init__()  # Properly initialize tk.Tk
        self.message = "saddeg " + name

    def printmessage(self):
        print(f"londoon {self.message}")


class Window(ShowTheThing):
    def __init__(self, name):
        super().__init__(name)  # Call parent __init__
        self.title("Main Window")
        self.geometry("400x200")
        print(f"subi: {name}")

class Vista(Window):
    def __init__(self, name):
        super().__init__(name)
    
    def prontmessage(self):
        print(self.message)


class Shape(Vista):
    print()

class Button(Shape):
    print()
    
    
class WelcomeView(Vista):
    actions = DoTheThing("party")
    def __init__(self, name, fump):
          super().__init__(name)
          self.prantmessage("uh")
          self.fump = fump


    
    
    def prantmessage(self, fimp):
        self.actions.sendnudes()
        print(f"knees{self.message, fimp}")


# Instantiate and Run
if __name__ == "__main__":
    main_window = Window("Hello, and welcome!")
    
    # Create and run WelcomeView
    welcome_view = WelcomeView("Grundig", "fumpo")
    welcome_view.printmessage()
    welcome_view.prontmessage()
    
    # Start the Tkinter main loop
    main_window.mainloop()





subi: Hello, and welcome!
subi: Grundig
sendnudes and budes
knees('saddeg Grundig', 'uh')
londoon saddeg Grundig
saddeg Grundig


I Kni what I'm doing now, just writ eht class heirarchy

In [ ]:
class App:
    print()

class Window:
    print()

class Pane(Window):
    print()

class Vista(Pane):
    print()






StudentAcrobat

In [2]:
import tkinter as tk
from tkinter import filedialog

def on_ctrl_u(event):
    print("Ctrl+u pressed: upe function triggered!")

    ErrorView("Oioi")

def on_ctrl_i(event):
    print("Ctrl+i pressed: ipe function triggered!")
    #root.destroy()

class NavigationPane(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, bg="lightgray")
        self.controller = controller

        self.title_label = tk.Label(self, text="", bg="lightgray", font=("Arial", 14))
        self.title_label.pack(side="right", padx=10)

        back_button = tk.Button(self, text="Back", command=self.controller.go_back)
        back_button.pack(side="left", padx=10)

    def update_title(self, title):
        self.title_label.config(text=title)

class View(tk.Frame):
    def __init__(self, parent, controller, bg_color):
        super().__init__(parent, bg=bg_color)
        self.controller = controller
        
# AlertView for handling errors
class AlertView(tk.Toplevel):
    def __init__(self, message, poiz):
        super().__init__()
        self.title("Alert")
        label = tk.Label(self, text=message, foreground="red")
        label.pack(padx=20, pady=20)
        close_button = tk.Button(self, text="Close", command=self.destroy)
        close_button.pack(pady=10)

class ErrorView(AlertView):
    name = "AlertView_Error"
    def __init__(self, master=None):
        super().__init__(master, "Error")
        #messagebox.showerror("Error", "An error has occurred!")
        print("longibbibibbies")
        self.title("Alert")
        label = tk.Label(self, text="Alert Errors")
        label.pack(pady=10)
        close_button = tk.Button(self, text="Close", command=self.destroy)
        close_button.pack(pady=10)

class Warning(AlertView):
    name = "AlertView_Warning"
    def __init__(self, master=None):
        super().__init__(master, "Warning")
        #messagebox.showwarning("Warning", "This is a warning!")
        self.title("Alert Warmomg")
        label = tk.Label(self, text="Alert Warnings")
        label.pack(pady=10)
        close_button = tk.Button(self, text="Close", command=self.destroy)
        close_button.pack(pady=10)

class Confirmation(AlertView):
    name = "AlertView_Confirmation"
    def __init__(self, master=None):
        super().__init__(master, "Confirmation")
        #messagebox.showinfo("Confirmation", "Action confirmed!")
        self.title("Alert Confirmation")
        label = tk.Label(self, text="Alert Confirmations")
        label.pack(pady=10)
        close_button = tk.Button(self, text="Close", command=self.destroy)
        close_button.pack(pady=10)

class Modal(tk.Toplevel):
    def __init__(self, parent):
        super().__init__(parent)
        self.transient(parent)
        self.grab_set()
        self.parent = parent
        self.title("Modal")

class WelcomeView(View):
    def __init__(self, parent, controller):
        super().__init__(parent, controller, bg_color="lightblue")
        label = tk.Label(self, text="Welcome to the App!", bg="lightblue")
        label.pack(pady=20)
        btn_menu = tk.Button(self, text="Go to Menu", command=lambda: controller.show_view("MenuView"))
        btn_menu.pack()

class MenuView(View):
    def __init__(self, parent, controller):
        super().__init__(parent, controller, bg_color="lightgreen")
        label = tk.Label(self, text="Menu", bg="lightgreen")
        label.pack(pady=20)
        btn_file_input = tk.Button(self, text="File Input", command=lambda: controller.show_view("FileInputView"))
        btn_file_input.pack()
        btn_data_output = tk.Button(self, text="Data Output", command=lambda: controller.show_view("DataOutputView"))
        btn_data_output.pack()

class FileInputView(View):
    def __init__(self, parent, controller):
        super().__init__(parent, controller, bg_color="lightyellow")
        label = tk.Label(self, text="File Input", bg="lightyellow")
        label.pack(pady=20)
        btn_browse = tk.Button(self, text="Browse File", command=self.open_file_browser)
        btn_browse.pack()
        btn_data_output = tk.Button(self, text="Data Output", command=lambda: controller.show_view("DataOutputView"))
        btn_data_output.pack()

    def open_file_browser(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            tk.Label(self, text=f"Selected File: {file_path}", bg="lightyellow").pack(pady=10)
        else:
            tk.Label(self, text="No file selected.", bg="lightyellow").pack(pady=10)

class DataOutputView(View):
    def __init__(self, parent, controller):
        super().__init__(parent, controller, bg_color="lightblue")
        label = tk.Label(self, text="Data Output", bg="grey")
        label.pack(pady=20)
        btn_data_output = tk.Button(self, text="Edit/Update", command=self.open_edit_update_modal)
        btn_data_output.pack()
        btn_data_output = tk.Button(self, text="Data Acrobatics", command=lambda: controller.show_view("StatVisView"))
        btn_data_output.pack()

    def open_edit_update_modal(self):
        EditUpdateView(self)


class EditUpdateView(Modal):
    def __init__(self, parent):
        super().__init__(parent)
        label = tk.Label(self, text="Edit/Update")
        label.pack(pady=20)
        btn_save = tk.Button(self, text="Save", command=self.destroy)
        btn_save.pack()
        btn_close = tk.Button(self, text="Close", command=self.destroy)
        btn_close.pack()

"""     def browse_file(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            tk.Label(self, text=f"Selected: {file_path}").pack(pady=10) """



        

class StatVisView(View):
    def __init__(self, parent, controller):
        super().__init__(parent, controller, bg_color="orange")
        label = tk.Label(self, text="Statistics/Visualisations", bg="pink")
        label.pack(pady=20)
        btn_data_output = tk.Button(self, text="See Acrobatics", command=self.open_display_Output_modal)
        btn_data_output.pack()

    def open_display_Output_modal(self):
        DisplayDataOutputsView(self)

class DisplayDataOutputsView(Modal):
    def __init__(self, parent):
        super().__init__(parent)
        label = tk.Label(self, text="Display Data Outputs")
        label.pack(pady=20)
        btn_save = tk.Button(self, text="Save", command=self.destroy)
        btn_save.pack()
        btn_close = tk.Button(self, text="Close", command=self.destroy)
        btn_close.pack()




# Shapes superclass and subclasses
class Shape:
    def draw(self, canvas):
        raise NotImplementedError("Subclasses must implement draw method")

class Circle(Shape):
    def draw(self, canvas):
        canvas.create_oval(50, 50, 150, 150, fill="blue", outline="black")

class Rectangle(Shape):
    def draw(self, canvas):
        canvas.create_rectangle(50, 50, 150, 100, fill="green", outline="black")

class BaseView:
    def __init__(self, root):
        self.root = root
        self.view_stack = []  # Stack to maintain navigation history as view names
        self.current_view = None

        self.navigation_pane = NavigationPane(root, self)
        self.navigation_pane.pack(side="top", fill="x")

        self.container = tk.Frame(root, bg="white")
        self.container.pack(expand=True, fill="both")

        self.views = {
            "WelcomeView": WelcomeView,
            "MenuView": MenuView,
            "FileInputView": FileInputView,
            "DataOutputView": DataOutputView,
            "StatVisView": StatVisView
        }
        print(" triggered!")
        self.show_view("WelcomeView")


    def add_view(self, view_name):
        if view_name in self.views:
            print()
        else:
            self.views.update({
                f"{view_name}": view_name
            })


    def gitHubAdded():
        print("GithubAdded")


    def show_view(self, view_name, push_to_stack=True):
        if self.current_view:
            if push_to_stack:
                # Push the name of the current view onto the stack
                self.view_stack.append(type(self.current_view).__name__)
            self.current_view.destroy()
        view_class = self.views.get(view_name)
        if view_class:
            self.current_view = view_class(self.container, self)
            self.current_view.pack(expand=True, fill="both")
            self.navigation_pane.update_title(view_name)

    def go_back(self):
        if self.view_stack:
            # Pop the name of the last view from the stack and show it
            last_view_name = self.view_stack.pop()
            self.show_view(last_view_name, push_to_stack=False)

if __name__ == "__main__":
    root = tk.Tk()
    root.title("App Structure Example with Navigation Pane")
    root.geometry("600x400")
    root.bind("<Control-u>", on_ctrl_u)  # Triggered by Ctrl+S
    root.bind("<Control-i>", on_ctrl_i)  # Triggered by Ctrl+Q
    app = BaseView(root)
    root.mainloop()


 triggered!


In [ ]:
import tkinter as tk
from tkinter import filedialog


class View(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller
        back_button = tk.Button(self, text="Back", command=self.controller.go_back)
        back_button.pack(anchor="nw", pady=10, padx=10)


class WelcomeView(View):
    def __init__(self, parent, controller):
        super().__init__(parent, controller)
        label = tk.Label(self, text="Welcome to the App!")
        label.pack(pady=20)
        btn_menu = tk.Button(self, text="Go to Menu", command=lambda: controller.show_view("MenuView"))
        btn_menu.pack()


class MenuView(View):
    def __init__(self, parent, controller):
        super().__init__(parent, controller)
        label = tk.Label(self, text="Menu")
        label.pack(pady=20)
        btn_file_input = tk.Button(self, text="File Input", command=lambda: controller.show_view("FileInputView"))
        btn_file_input.pack()


class FileInputView(View):
    def __init__(self, parent, controller):
        super().__init__(parent, controller)
        label = tk.Label(self, text="File Input")
        label.pack(pady=20)
        btn_browse = tk.Button(self, text="Browse File", command=self.open_file_browser)
        btn_browse.pack()

    def open_file_browser(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            tk.Label(self, text=f"Selected File: {file_path}").pack(pady=10)
        else:
            tk.Label(self, text="No file selected.").pack(pady=10)

# Shapes superclass and subclasses
class Shape:
    def draw(self, canvas):
        raise NotImplementedError("Subclasses must implement draw method")

class Circle(Shape):
    def draw(self, canvas):
        canvas.create_oval(50, 50, 150, 150, fill="blue", outline="black")

class Rectangle(Shape):
    def draw(self, canvas):
        canvas.create_rectangle(50, 50, 150, 100, fill="green", outline="black")

class NavigationPane(View):
    def __init__(self, parent, controller):
        super().__init__(parent, controller)
        label = tk.Label(self, text="Shapes Navigation Pane")
        label.pack(pady=20)

        btn_circle = tk.Button(self, text="Draw Circle", command=self.draw_circle)
        btn_circle.pack()

        btn_rectangle = tk.Button(self, text="Draw Rectangle", command=self.draw_rectangle)
        btn_rectangle.pack()

        self.canvas = tk.Canvas(self, width=200, height=200, bg="white")
        self.canvas.pack(pady=20)

    def draw_circle(self):
        self.canvas.delete("all")  # Clear the canvas
        circle = Circle()
        circle.draw(self.canvas)

    def draw_rectangle(self):
        self.canvas.delete("all")  # Clear the canvas
        rectangle = Rectangle()
        rectangle.draw(self.canvas)

class BaseView:
    def __init__(self, root):
        self.root = root
        self.view_stack = []  # Stack to maintain navigation history as view names
        self.current_view = None
        self.views = {
            "WelcomeView": WelcomeView,
            "MenuView": MenuView,
            "FileInputView": FileInputView,
            "NavigationPane": NavigationPane,
        }
        self.show_view("WelcomeView")

    def show_view(self, view_name, push_to_stack=True):
        if self.current_view:
            if push_to_stack:
                # Push the name of the current view onto the stack
                self.view_stack.append(type(self.current_view).__name__)
            self.current_view.destroy()
        view_class = self.views.get(view_name)
        if view_class:
            self.current_view = view_class(self.root, self)
            self.current_view.pack(expand=True, fill="both")

    def go_back(self):
        if self.view_stack:
            # Pop the name of the last view from the stack and show it
            last_view_name = self.view_stack.pop()
            self.show_view(last_view_name, push_to_stack=False)

if __name__ == "__main__":
    root = tk.Tk()
    root.title("App Structure Example with Stack Navigation and Shapes")
    root.geometry("600x400")
    app = BaseView(root)
    root.mainloop()


2024-12-18 22:48:32.740 python[738:6267] +[CATransaction synchronize] called within transaction
2024-12-18 22:48:33.190 python[738:6267] +[CATransaction synchronize] called within transaction
